In [ ]:
## Update matplotlib to a version that can label bar graphs.
#!pip install -U matplotlib --user

In [ ]:
%matplotlib inline

import sympy as sp
sp.init_printing(use_latex ='mathjax')
import scipy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lmfit as lf
import os
import seaborn as sns
from myheatmap import myheatmap
from lmfit import Model
import lmfit
from tabulate import tabulate
import matplotlib
import math 
from scipy.signal import find_peaks

verbose = False

sns.set_context('talk')

matplotlib.__version__ ## Need version 3.5 for labeled bargraphs. otherwise set labelbarchart = False below

In [ ]:
#Define all variables for sympy

#individual springs that correspond to individual masses
k1 = sp.symbols('k_1', real = True)
k2 = sp.symbols('k_2', real = True)

#springs that connect two masses
k12 = sp.symbols('k_12', real = True)

#damping coefficients
b1 = sp.symbols('b1', real = True)
b2 = sp.symbols('b2', real = True)
 
#masses
m1 = sp.symbols('m1', real = True)
m2 = sp.symbols('m2', real = True)

#Driving force amplitude
F = sp.symbols('F', real = True)

#driving frequency (leave as variable)
wd = sp.symbols('\omega_d', real = True)

In [ ]:
def syserror(x_found,x_set):
    return abs(x_found-x_set)/x_set

def combinedsyserror(syserrors, notdof): # notdof = not degrees of freedom, meaning the count of fixed parameters.
    dof = len(syserrors) - notdof
    squared = [(err**2) for err in syserrors]
    rms = np.sqrt(sum(squared) / dof)
    avg = sum(syserrors)/ dof
    return avg, rms, max(syserrors)

def complexamp(A,phi):
    return A * np.exp(1j*phi)

def rsqrd(model, data, plot=False, x=None, newfigure = True):
    SSres = sum((data - model)**2)
    SStot = sum((data - np.mean(data))**2)
    rsqrd = 1 - (SSres/ SStot)
    
    if plot:
        if newfigure:
            plt.figure()
        plt.plot(x,data, 'o')
        plt.plot(x, model, '--')
    
    return rsqrd

# For driven, damped oscillator: res_freq = sqrt(k/m - b^2/(2m^2))
# Note: Requires b < sqrt(2mk) to be significantly underdamped
# Otherwise there is no resonant frequency and we get an error from the negative number under the square root
# This works for monomer and for weak coupling. It does not work for strong coupling.
def approx_res_freq(k, m, b):
    return math.sqrt(k/m - (b*b)/(2*m*m))

## source: https://en.wikipedia.org/wiki/Q_factor#Mechanical_systems
def approx_Q(k, m, b):
    return math.sqrt(m*k)/b

def approx_width(k, m, b):
    return approx_res_freq(k, m, b) / approx_Q(k, m, b)

def quadratic_formula(a, b, c):
    return (-b + math.sqrt(b*b - 4*a*c))/(2*a), (-b - math.sqrt(b*b - 4*a*c))/(2*a)

In [ ]:
#symbolically Solve for driving amplitudes and phase using sympy

#Matrix for complex equations of motion
driven = sp.Matrix([[-wd**2*m1 + 1j*wd*b1 + k1 + k12, -k12], [-k12, -wd**2*m2 + 
  1j*wd*b2 + k2 + k12]])

#Matrices for Cramer's Rule
driven_m1 = sp.Matrix([[F, -k12], [0, -wd**2*m2 + 1j*wd*b2 + k2 + k12]])

driven_m2 = sp.Matrix([[-wd**2*m1 + 1j*wd*b1 + k1 + k12, F], [-k12, 0]])

#Apply Cramer's Rule
complexamp1, complexamp2 = (driven_m1.det()/driven.det(), driven_m2.det()/driven.det())

#Solve for phases for each mass
delta1 = sp.arg(complexamp1) # Returns the argument (phase angle in radians) of a complex number. 
delta2 = sp.arg(complexamp2) # sp.re(complexamp2)/sp.cos(delta2) (this is the same thing)

#Wrap phases for plots

wrap1 = (delta1)%(2*sp.pi)
wrap2 = (delta2)%(2*sp.pi)

#Solve for amplitude coefficients
amp1 = sp.Abs(complexamp1)
amp2 = sp.Abs(complexamp2)

"""complexamp1 = amp1 * sp.exp(sp.I * sp.pi * delta1)
complexamp2 = amp2 * sp.exp(sp.I * sp.pi * delta2) """

if verbose: # display symbolic solutions
    display(r"R1 Amplitude:")
    display(amp1)

    display(r"R2 Amplitude:")
    display(amp2)

    display(r"R1 complex amplitude:")
    display(complexamp1)
    
    display(r"R2 complex amplitude:")
    display(complexamp2)
    
    display("R1 Real amplitude:")
    display(sp.re(complexamp1))
    
    display("R1 Imaginary amplitude:")
    display(sp.im(complexamp1))
    
    display("R2 Real amplitude:")
    display(sp.re(complexamp2))
    
    display("R2 Imaginary amplitude:")
    display(sp.im(complexamp2))

#lambdify curves using sympy

c1 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), amp1)
t1 = sp.lambdify((wd, k1, k2, k12, b1, b2, F,  m1, m2), wrap1)

c2 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), amp2)
t2 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), wrap2)

re1 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), sp.re(complexamp1))
im1 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), sp.im(complexamp1))
re2 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), sp.re(complexamp2))
im2 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), sp.im(complexamp2))

#define functions

#curve = amplitude, theta = phase, e = error (i.e. noise)
def curve1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return c1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def theta1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return t1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) - 2*np.pi + e
    
def curve2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return c2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def theta2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return t2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) - 2*np.pi + e
    
def realamp1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return re1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def imamp1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return im1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def realamp2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return re2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def imamp2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return im2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e

In [ ]:
#Use functions to make matrices of amplitude and phase for each resonator (with noise)

MONOMER = False

#define set values
m1_set = 5
m2_set = 3
b1_set = 1
b2_set = 0.5
k1_set = 12
k2_set = 27
k12_set = 1
F_set = 1


print('Approximate Q1: ' + "{:.2f}".format(approx_Q(k = k1_set, m = m1_set, b=b1_set)) + 
      ' width: ' + "{:.2f}".format(approx_width(k = k1_set, m = m1_set, b=b1_set)))
print('Approximate Q2: ' + "{:.2f}".format(approx_Q(k = k2_set, m = m2_set, b=b2_set)) +
      ' width: ' + "{:.2f}".format(approx_width(k = k2_set, m = m2_set, b=b2_set)))

if MONOMER:
    k12_set = 0 # overwrite the coupling
    vals_set = [m1_set,  b1_set,  k1_set,  F_set]
else:
    vals_set = [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set]

#define number of samples (i.e. range over which to vary the frequency); 30 to test, 200 is very nice.
n = 30

noiselevel = 1
use_complexnoise = True

amplitudenoisefactor1 = 0.005
amplitudenoisefactor2 = 0.0005
phasenoisefactor1 = 0.1
phasenoisefactor2 = 0.2
complexamplitudenoisefactor = 0.0005

#define noise (randn(n,) gives a array of normally-distributed random numbers of size n)
def amp1_noise(n): 
    return noiselevel* amplitudenoisefactor1 * np.random.randn(n,)
def phase1_noise(n):
    return noiselevel* phasenoisefactor1 * np.random.randn(n,)
def amp2_noise(n):
    return noiselevel* amplitudenoisefactor2 * np.random.randn(n,)
def phase2_noise(n):
    return noiselevel* phasenoisefactor2 * np.random.randn(n,)
def complex_noise(n):
    return noiselevel* complexamplitudenoisefactor * np.random.randn(n,)

#define driving frequency range (gives array of n evenly spaced numbers between 0.1 and 5)
drive = np.linspace(0.1, 5, num = n)

R1_amp_noiseless = curve1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R1_phase_noiseless = theta1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R2_amp_noiseless = curve2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R2_phase_noiseless = theta2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)

R1_real_amp_noiseless = realamp1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R1_im_amp_noiseless = imamp1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R2_real_amp_noiseless = realamp2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R2_im_amp_noiseless = imamp2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)

def amp(a,b):
    return np.sqrt(a**2 + b**2)

usenoise = True

## Calculate the amplitude and phase as spectra, possibly adding noise
def calculate_spectra(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, n):
    if usenoise: # add a random vector of positive and negative numbers to the curve.

        if use_complexnoise: # apply noise in cartesian coordinates
            R1_real_amp = realamp1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, complex_noise(n))
            R1_im_amp   = imamp1  (drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, complex_noise(n))
            R2_real_amp = realamp2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, complex_noise(n))
            R2_im_amp   = imamp2  (drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, complex_noise(n))

            R1_amp   = amp(R1_real_amp, R1_im_amp)
            R2_amp   = amp(R2_real_amp, R2_im_amp)
            R1_phase = np.unwrap(np.angle(R1_real_amp + R1_im_amp*1j))
            R2_phase = np.unwrap(np.angle(R2_real_amp + R2_im_amp*1j))

        else: # apply noise in polar coordinates
            R1_amp   = curve1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, amp1_noise(n))
            R1_phase = theta1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, phase1_noise(n))
            R2_amp   = curve2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, amp2_noise(n))
            R2_phase = theta2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, phase2_noise(n))

            R1_complexamp = complexamp(R1_amp, R1_phase)
            R2_complexamp = complexamp(R2_amp, R2_phase)

            R1_real_amp = np.real(R1_complexamp)
            R1_im_amp   = np.imag(R1_complexamp)
            R2_real_amp = np.real(R2_complexamp)
            R2_im_amp   = np.imag(R2_complexamp)

    else:
        R1_amp = R1_amp_noiseless
        R1_phase = R1_phase_noiseless
        R2_amp = R2_amp_noiseless
        R2_phase = R2_phase_noiseless
        R1_real_amp = R1_real_amp_noiseless
        R1_im_amp = R1_im_amp_noiseless
        R2_real_amp = R2_real_amp_noiseless
        R2_im_amp = R2_im_amp_noiseless
    
    return R1_amp, R1_phase, R2_amp, R2_phase, R1_real_amp, R1_im_amp, R2_real_amp, R2_im_amp

## actually calculate the spectra
R1_amp, R1_phase, R2_amp, R2_phase, R1_real_amp, R1_im_amp, R2_real_amp, R2_im_amp = \
    calculate_spectra(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, n)
    
def listlength(list1):
    try:
        length = len(list1)
    except TypeError:
        length = 1
    return length

## Calculate the amplitude and phase as individual measurements, possibly adding noise
if use_complexnoise:
    # Unfortunately doesn't yet work with multiple drive frequencies
    def noisyR1ampphase(drive, vals_set=vals_set, noiselevel = noiselevel):
        if MONOMER:
            k12_set = 0 # overwrite the coupling
            [m1_set,  b1_set,  k1_set,  F_set] = vals_set
        else:
            [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set] = vals_set
        n = listlength(drive)
        ## calculate fresh the noise of amplitude 1. This is an independent noise calculation.
        R1_real_amp = realamp1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* complexamplitudenoisefactor * np.random.randn(n,))
        R1_im_amp   = imamp1  (drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* complexamplitudenoisefactor * np.random.randn(n,))
        R1complexamp = R1_real_amp + 1j * R1_im_amp
        return amp(R1_real_amp,R1_im_amp), np.angle(R1_real_amp + R1_im_amp*1j), R1complexamp

    def noisyR2ampphase(drive, vals_set=vals_set, noiselevel = noiselevel):
        if MONOMER:
            k12_set = 0 # overwrite the coupling
            [m1_set,  b1_set,  k1_set,  F_set] = vals_set
        else:
            [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set] = vals_set
        n = listlength(drive)
        ## calculate fresh the noise of amplitude. This is an independent noise calculation.
        R2_real_amp = realamp2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* complexamplitudenoisefactor * np.random.randn(n,))
        R2_im_amp   = imamp2  (drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* complexamplitudenoisefactor * np.random.randn(n,))
        R2complexamp = R2_real_amp + 1j * R2_im_amp
        return amp(R2_real_amp,R2_im_amp), np.angle(R2_real_amp + R2_im_amp*1j), R2complexamp

else:
    def noisyR1ampphase(drive, vals_set=vals_set, noiselevel = noiselevel):
        if MONOMER:
            k12_set = 0 # overwrite the coupling
            [m1_set,  b1_set,  k1_set,  F_set] = vals_set
        else:
            [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set] = vals_set
        n = listlength(drive)
        a = curve1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* amplitudenoisefactor1 * np.random.randn(n,))
        p = theta1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* phasenoisefactor1 * np.random.randn(n,))
        return a,p, complexamp(a,p)

    def noisyR2ampphase(drive, vals_set=vals_set, noiselevel = noiselevel):
        if MONOMER:
            k12_set = 0 # overwrite the coupling
            [m1_set,  b1_set,  k1_set,  F_set] = vals_set
        else:
            [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set] = vals_set
        n = listlength(drive)
        a = curve2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* amplitudenoisefactor2 * np.random.randn(n,))
        p = theta2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* phasenoisefactor2 * np.random.randn(n,))
        return a,p, complexamp(a,p)
    
res1 = approx_res_freq(k1_set, m1_set, b1_set)
res2 = approx_res_freq(k2_set, m2_set, b2_set)

if MONOMER:
    print('Resonant frequency expected at approximately ' 
          + str(res1))
else:
    print('Resonant frequencies expected at approximately ' 
          + "{:.2f}".format(res1) + ' and ' 
          + "{:.2f}".format(res2))
    
""" Given a limited set of available frequencies called "drive", 
find those indices that most closely correspond to the desired frequencies. 
This will not throw an error if two are the same; that could be added by checking if nunique is a shorter length. """
def freqpoints(desiredfreqs, drive = drive):
    p = [] # p stands for frequency points; these are the indicies of frequencies that we will be measuring.
    for f in desiredfreqs:
            absolute_val_array = np.abs(drive - f)
            f_index = absolute_val_array.argmin()
            p.append(f_index)
    return p

In [ ]:
#Choose points for SVD

desiredfreqs = [res1, res2]
#desiredfreqs = [2., 2.4]

p = freqpoints(desiredfreqs, drive)

#p = [14,15,16,29,30,31] #indices of frequencies to use
#p=[14,16]
#p = range(len(drive))
print(p)

def SNRcalc(freq, plot = False, ax = None, vals_set=vals_set, noiselevel = noiselevel, detailed = False):
    n = 50 # number of randomized values to calculate
    amps1 = np.zeros(n)
    zs1 = np.zeros(n ,dtype=complex)
    amps2 = np.zeros(n)
    zs2 = np.zeros(n ,dtype=complex)
    for j in range(n):
        thisamp1, _, thisz1 = noisyR1ampphase(freq, vals_set=vals_set, noiselevel = noiselevel)
        amps1[j] = thisamp1
        zs1[j] = thisz1[0] # multiple simulated measurements of complex amplitude Z1 (of R1)
        thisamp2, _, thisz2 = noisyR2ampphase(freq, vals_set=vals_set, noiselevel = noiselevel)
        amps2[j] = thisamp2
        zs2[j] = thisz2[0] # multiple simulated measurements of complex amplitude Z2 (of R2)
    SNR_R1 = np.mean(amps1) / np.std(amps1)
    SNR_R2 = np.mean(amps2) / np.std(amps2)

    if plot:
        if ax is not None:
            plt.sca(ax)
        plt.plot(np.real(zs1), np.imag(zs1), '.', alpha = .2) 
        plt.plot(np.real(zs2), np.imag(zs2), '.', alpha = .2) 
        plt.plot(0,0, 'o')
        plt.gca().axis('equal');
        plt.title('Freq: ' + str(freq) +   
                  ', SNR R1: ' + '{num:.{dig}g}'.format(num=SNR_R1, dig=3) + 
                  ', SNR R2: ' + '{num:.{dig}g}'.format(num=SNR_R2, dig=3))
    if detailed:
        return SNR_R1,SNR_R2, np.mean(amps1), np.std(amps1),  np.mean(amps2), np.std(amps2)
    else:
        return SNR_R1,SNR_R2

#SNRcalc(drive[p[1]], plot = True)

table = []
for i in range(len(p)):
    SNR1, SNR2 = SNRcalc(drive[p[i]])
    table.append([drive[p[i]], R1_amp[p[i]], R1_phase[p[i]], R2_amp[p[i]], R2_phase[p[i]], 
                  complexamp(R1_amp[p[i]],R1_phase[p[i]] ),
                  complexamp(R2_amp[p[i]], R2_phase[p[i]]),
                  SNR1, SNR2,
                 syserror(R1_amp[p[i]], R1_amp_noiseless[p[i]]),
                 (R1_phase[p[i]] - R1_phase_noiseless[p[i]]),
                 syserror(R2_amp[p[i]],R2_amp_noiseless[p[i]]),
                 (R2_phase[p[i]]-R2_phase_noiseless[p[i]]),
                 ])
                 
    
df = pd.DataFrame(data = table, columns = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase',
                                           'R1AmpCom', 'R2AmpCom',
                                           'SNR_R1','SNR_R2',
                                           'R1Amp_syserror', 'R1Phase_diff', 'R2Amp_syserror', 'R2Phase_diff'])

df

In [ ]:
#Plots of simulated spectra

print('Simulated spectra data, as if from experiment')
print('Approximate Q1: ' + "{:.2f}".format(approx_Q(k = k1_set, m = m1_set, b=b1_set)) + 
      ' width: ' + "{:.2f}".format(approx_width(k = k1_set, m = m1_set, b=b1_set)))
print('Approximate Q2: ' + "{:.2f}".format(approx_Q(k = k2_set, m = m2_set, b=b2_set)) +
      ' width: ' + "{:.2f}".format(approx_width(k = k2_set, m = m2_set, b=b2_set)))
if MONOMER:
    print('Resonant frequency expected at approximately ' 
          + str(res1))
else:
    print('Resonant frequencies expected at approximately ' 
          + str(res1) + ' and ' 
          + str(res2))

morefrequencies = np.linspace(0.1, 5, num = n*10)

fig, ((ax1, ax3),(ax2,ax4),(ax5, ax6)) = plt.subplots(3,2, figsize = (10,10))

ax1.plot(morefrequencies, curve1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), # true curve
         color = 'gray', alpha = 0.2)
ax1.plot(drive, R1_amp, '.', color = 'lightsteelblue') # noisy simulated data
ax1.set_ylabel('Amplitude\n')
ax1.set_title('Simulated R1 Amp')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax1.plot(df.drive[i], df.R1Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
ax2.plot(morefrequencies, theta1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set,  0), # true curve
         color = 'gray', alpha = 0.2)        
ax2.plot(drive, R1_phase, '.', color = 'lightsteelblue') # noisy simulated data
ax2.set_ylabel('Phase (rad)')
ax2.set_title('Simulated R1 Phase')

#For loop to plot chosen values from table
for i in range(df.shape[0]):
    ax2.plot(df.drive[i], df.R1Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
ax3.plot(morefrequencies, curve2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), # true curve
         color = 'gray', alpha = 0.2)    
ax3.plot(drive, R2_amp, '.', color = 'lightsteelblue')
ax3.set_ylabel('Amplitude\n')
ax3.set_title('Simulated R2 Amp')

#For loop to plot chosen values from table
for i in range(df.shape[0]):
    ax3.plot(df.drive[i], df.R2Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
    
ax4.plot(morefrequencies, theta2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set,  0), # true curve
         color = 'gray', alpha = 0.2)
ax4.plot(drive, R2_phase, '.', color = 'lightsteelblue')
ax4.set_ylabel('Phase (rad)')
ax4.set_title('Simulated R2 Phase')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax4.plot(df.drive[i], df.R2Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
for ax in [ax1,ax2,ax3,ax4]:
    plt.sca(ax)
    plt.xticks([res1, res2])
    ax.set_xlabel('Freq (rad/s)')

    
plt.tight_layout()

def plotcomplex(complexZ, parameter, title = 'Complex Amplitude', cbar_label='Frequency (rad/s)', label_markers=[], ax=plt.gca(), s=50, cmap = 'magma'):
    plt.sca(ax)
    sc = ax.scatter(np.real(complexZ), np.imag(complexZ), s=s, c = parameter, cmap = cmap ) # s is marker size
    cbar = plt.colorbar(sc)
    cbar.outline.set_visible(False)
    cbar.set_label(cbar_label)
    ax.set_xlabel('Real Amplitude')
    ax.set_ylabel('Imaginary Amplitude')
    ax.axis('equal');
    plt.title(title)
    plt.gcf().canvas.draw()
    ymin, ymax = ax.get_ylim()
    xmin, xmax = ax.get_xlim()
    plt.vlines(0, ymin=ymin, ymax = ymax, colors = 'k', linestyle='solid', alpha = .5)
    plt.hlines(0, xmin=xmin, xmax = xmax, colors = 'k', linestyle='solid', alpha = .5)
    #ax.plot([0,1],[0,0], lw=10,transform=ax.xaxis.get_transform() )#,transform=ax.xaxis.get_transform() ) #transform=ax.transAxes
    
    # label markers that are closest to the desired frequencies
    for label in label_markers:
        absolute_val_array = np.abs(drive - label)
        label_index = absolute_val_array.argmin()
        closest_element = parameter[label_index]
        plt.annotate(text=str(round(closest_element,2)), 
                     xy=(np.real(complexZ[label_index]), np.imag(complexZ[label_index])) )

#Calculate complex amplitudes of spectra

Z1 = complexamp(R1_amp, R1_phase)
Z2 = complexamp(R2_amp, R2_phase)

#fig, (ax5, ax6) = plt.subplots(1, 2, figsize = (10,5))

# true curves
ax5.plot(realamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         imamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         color='gray', alpha = .5)
ax6.plot(realamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         imamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         color='gray', alpha = .5)

plotcomplex(Z1, drive, 'Complex Amplitude Z1', ax=ax5, label_markers=[res1,res2])
ax5.scatter(np.real(df.R1AmpCom), np.imag(df.R1AmpCom), s=150, facecolors='none', edgecolors='k') 
plotcomplex(Z2, drive, 'Complex Amplitude Z2', ax=ax6, label_markers=[res1,res2])
ax6.scatter(np.real(df.R2AmpCom), np.imag(df.R2AmpCom), s=150, facecolors='none', edgecolors='k') 
      
plt.tight_layout()

In [ ]:
df

In [ ]:
#Singular Value Decomposition

verbose = True

elementnames = ['m1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force']

"""Zmatrix2resonators(df) will return a matrix for svd for any number of frequency measurements, 
listed in each row of the dataframe measurementdf """
def Zmatrix2resonators(measurementdf, 
                       frequencycolumn = 'drive', 
                       complexamplitude1 = 'R1AmpCom', complexamplitude2 = 'R2AmpCom', dtype=np.double):
    Zmatrix = []
    for rowindex in measurementdf.index:
        w = measurementdf[frequencycolumn][rowindex]
        #print(w)
        ZZ1 = measurementdf[complexamplitude1][rowindex]
        ZZ2 = measurementdf[complexamplitude2][rowindex]
        Zmatrix.append([-w**2*np.real(ZZ1), 0, -w*np.imag(ZZ1), 0, np.real(ZZ1), 0, np.real(ZZ1)-np.real(ZZ2), -1])
        Zmatrix.append([-w**2*np.imag(ZZ1), 0, w*np.real(ZZ1), 0, np.imag(ZZ1), 0, np.imag(ZZ1)-np.imag(ZZ2), 0])
        Zmatrix.append([0, -w**2*np.real(ZZ2), 0, -w*np.imag(ZZ2), 0, np.real(ZZ2), np.real(ZZ2)-np.real(ZZ1), 0])
        Zmatrix.append([0, -w**2*np.imag(ZZ2), 0, w*np.real(ZZ2), 0, np.imag(ZZ2), np.imag(ZZ2)-np.imag(ZZ1), 0])
    #display(Zmatrix)
    return np.array(Zmatrix, dtype=dtype)

Zmatrix = Zmatrix2resonators(df, frequencycolumn = 'drive', complexamplitude1 = 'R1AmpCom', complexamplitude2 = 'R2AmpCom')

if verbose:
    print("Z matrix:") 
    display(pd.DataFrame(Zmatrix, columns = elementnames))

#SVD
u, s, vh = np.linalg.svd(Zmatrix, full_matrices = True)
#u, s, vh = sc.linalg.svd(Zmatrix, full_matrices = False, lapack_driver = 'gesvd')

# the smallest singular value is always the last one, index -1
print('Singular values:')
print(s)

plt.semilogy(s, '.')
plt.axhline(10**-2, color='gray')
plt.title('singular values');

In [ ]:
# singular vectors. # M1, M2, B1, B2, K1, K2, K12, FD
vh

In [ ]:
print("Checking the python-normalization")

vect =vh[-1]
sq = [el**2 for el in vect]
sum(sq)

In [ ]:
[M1, M2, B1, B2, K1, K2, K12, FD] = vh[-1]

vals = [['Norm m1', 'Norm m2', 'Norm b1', 'Norm b2', 'Norm k1', 'Norm k2','Norm c12', 'Norm Driving Force']]
vals.append((M1, M2, B1, B2, K1, K2, K12, FD))

#print("Python-Normalized singular vector corresponding to elements vector")
#print(tabulate(vals, headers = 'firstrow', tablefmt = 'fancy_grid'))

In [ ]:
#View vh array and assign variables to proper row vector

print('Results of analysis versus set values:')
print("The python normalization loses one degree of freedom, \
so we must be able to independently determine one element; say it's the driving force")

""" 1d nullspace normalization """
def normalize_elements_1d_by_force(unnormalizedelements, F_set=F_set):
    # elements vector: 'm1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force'
    c = F_set / unnormalizedelements[-1]
    elements = [c*unnormalizedelements[k] for k in range(len(unnormalizedelements)) ]
    return elements

M1, M2, B1, B2, K1, K2, K12, FD = normalize_elements_1d_by_force([M1, M2, B1, B2, K1, K2, K12, FD], F_set)
if MONOMER:
    elements = [M1,  B1,  K1,  FD]
else:
    elements = [M1, M2, B1, B2, K1, K2, K12, FD]
#elements = [element.real for element in elements if element.imag == 0 ]
syserrors = [syserror(elements[i], vals_set[i]) for i in range(len(elements))]
avg1d, rms1d, max1d = combinedsyserror(syserrors, 1)

if MONOMER:
    elementlist = [['m1','b1', 'k1', 'Driving Force']]
else:
    elementlist = [['m1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force']]
vals2 = elementlist.copy() # initialize elementsdf
vals2.append(elements)
vals2.append(vals_set)
vals2.append(syserrors)

#print(tabulate(vals2, headers = 'firstrow', tablefmt = 'fancy_grid'))
elementsdf = pd.DataFrame(vals2, index = ['labels', 'from_svd', 'set', "syserror"])
new_header = elementsdf.iloc[0] 
elementsdf.columns = new_header
print("How good are the 1D nullspace results?")
display(elementsdf)
elementsdf1d = elementsdf.copy()

#Calculate Rsqrd compared to 1D nullspace to test the 1D nullspace model

plotting = False
amp1_rsqrd = rsqrd(model=curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), data=R1_amp, plot=plotting, x=drive)
phase1_rsqrd = rsqrd(data=R1_phase, model=theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
amp2_rsqrd = rsqrd(data=R2_amp, model=curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
phase2_rsqrd = rsqrd(data = R2_phase, model= theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)


## Calculate systematic error to test the 1D nullspace results

print('Noise level: ' + str(noiselevel))
print('R1 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R1.min(), dig=3) + ' to ' 
      + '{num:.{dig}g}'.format(num=df.SNR_R1.max(), dig=3))
if not MONOMER:
    print('R2 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R2.min(), dig=3) + ' to ' 
          + '{num:.{dig}g}'.format(num=df.SNR_R2.max(), dig=3))
print('Average systematic error: ', '{num:.{dig}g}'.format(num=avg1d, dig=3))
print('RMS systematic error: ', '{num:.{dig}g}'.format(num=rms1d, dig=3))
print('Max systematic error: ', '{num:.{dig}g}'.format(num=max1d, dig=3))



labelbarchart = False

fig,ax = plt.subplots()
if labelbarchart:
    bargraph = ax.bar(elementsdf.columns,elementsdf.transpose()['syserror']*100)
    plt.gca().bar_label(bargraph)
else:
    (elementsdf.transpose()['syserror']*100).plot(kind='bar')
plt.gca().set_ylabel('systematic error (%)\n')
plt.gca().set_xlabel('');
plt.title('1D nullspace')
plt.show()

print("R1 Amp Rsqrd = ", amp1_rsqrd,
     "\nR1 Phase Rsqrd = ", phase1_rsqrd)
if not MONOMER:
      print("R2 Amp Rsqrd = ", amp2_rsqrd,
          "\nR2 Phase Rsqrd = ", phase2_rsqrd)

In [ ]:
## what if the null-space is 2D?

#View vh array and assign variables to proper row vector

print('Results of analysis versus set values:')

print("If the null-space is 2D, we must be able to independently determine two elements; say we know resonating frequencies and force")

def normalize_elements_to_res1_and_F_2d(vh, vals_set = vals_set):
    
    if MONOMER:
        [m1_set,  b1_set,  k1_set,  F_set] = vals_set
    else:
        [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set] = vals_set
    
    # elements vector: 'm1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force'
    vect1 = vh[-1]
    [m1_1, m2_1, b1_1, b2_1, k1_1, k2_1, c12_1, F_1] = vect1
    vect2 = vh[-2]
    [m1_2, m2_2, b1_2, b2_2, k1_2, k2_2, c12_2, F_2] = vect2
    
    # Assume we know the resonant frequency of one of the driven, damped oscillators
    # We seem to get much more accurate results knowing the first one, perhaps because that one is being driven
    res_freq1 = approx_res_freq(k1_set, m1_set, b1_set)
    #res_freq2 = approx_res_freq(k2_set, m2_set, b2_set)
    
    # Subscript by 1 and 2 for the two null space vectors
    # Then in res_freq formula, substitute k -> k_1 + Rk_2, m -> m_1 + Rm_2, b -> b_1 + Rb_2
    # Solve for R, the weight of null vector 2 relative to null vector 1 (formula found using Mathematica)
    # The formula is quadratic, so we get two values of R for each oscillator
    # Pick the one that gives the correct (or closer to correct) resonating frequency
    # For simplicity, first solve for A, B, C, the coefficients of the quadratic equation
    
    osc1_A = -b1_2**2 + 2 * k1_2 * m1_2 - 2 * m1_2**2 * res_freq1**2
    osc1_B = -2 * b1_1 * b1_2 + 2 * k1_2 * m1_1 + 2 * k1_1 * m1_2 - 4 * m1_1 * m1_2 * res_freq1**2
    osc1_C = -b1_1**2 + 2 * k1_1 * m1_1 - 2 * m1_1**2 * res_freq1**2
    # If there's a ValueError, just do 1D
    osc1_R1, osc1_R2 = 0, 0
    try:
        osc1_R1, osc1_R2 = quadratic_formula(osc1_A, osc1_B, osc1_C)
    except ValueError:
        pass
    
    # There may be ValueErrors if there exists no resonating frequency for the incorrect R
    # In that case, we make the difference infinity so that R isn't chosen
    osc1_R1_diff = float('inf')
    osc1_R2_diff = float('inf')
    try:
        osc1_R1_diff = abs(approx_res_freq(k1_1 + osc1_R1 * k1_2, m1_1 + osc1_R1 * m1_2, b1_1 + osc1_R1 * b1_2) - res_freq1)
    except ValueError:
        pass
    try:
        osc1_R2_diff = abs(approx_res_freq(k1_1 + osc1_R2 * k1_2, m1_1 + osc1_R2 * m1_2, b1_1 + osc1_R2 * b1_2) - res_freq1)
    except ValueError:
        pass
    osc1_R = osc1_R1 if osc1_R1_diff < osc1_R2_diff else osc1_R2
    
    #osc2_A = -b2_2**2 + 2 * k2_2 * m2_2 - 2 * m2_2**2 * res_freq2**2
    #osc2_B = -2 * b2_1 * b2_2 + 2 * k2_2 * m2_1 + 2 * k2_1 * m2_2 - 4 * m2_1 * m2_2 * res_freq2**2
    #osc2_C = -b2_1**2 + 2 * k2_1 * m2_1 - 2 * m2_1**2 * res_freq2**2
    #osc2_R1, osc2_R2 = quadratic_formula(osc2_A, osc2_B, osc2_C)
    
    #osc2_R1_diff = float('inf')
    #osc2_R2_diff = float('inf')
    #try:
    #    osc2_R1_diff = abs(approx_res_freq(k2_1 + osc2_R1 * k2_2, m2_1 + osc2_R1 * m2_2, b2_1 + osc2_R1 * b2_2) - res_freq2)
    #except ValueError:
    #    pass
    #try:
    #    osc2_R2_diff = abs(approx_res_freq(k2_1 + osc2_R2 * k2_2, m2_1 + osc2_R2 * m2_2, b2_1 + osc2_R2 * b2_2) - res_freq2)
    #except ValueError:
    #    pass
    #osc2_R = osc2_R1 if osc2_R1_diff < osc2_R2_diff else osc2_R2
    
    # For testing purposes
    #calc_res_freq_1with1 = approx_res_freq(k1_1 + osc1_R * k1_2, m1_1 + osc1_R * m1_2, b1_1 + osc1_R * b1_2)
    #calc_res_freq_1with2 = approx_res_freq(k1_1 + osc2_R * k1_2, m1_1 + osc2_R * m1_2, b1_1 + osc2_R * b1_2)
    #calc_res_freq_2with1 = approx_res_freq(k2_1 + osc1_R * k2_2, m2_1 + osc1_R * m2_2, b2_1 + osc1_R * b2_2)
    #calc_res_freq_2with2 = approx_res_freq(k2_1 + osc2_R * k2_2, m2_1 + osc2_R * m2_2, b2_1 + osc2_R * b2_2)
    #print("Actual Oscillator 1 Resonant Frequency: " + str(res_freq1))
    #print("Calculated Oscillator 1with1 Resonant Frequency: " + str(calc_res_freq_1with1))
    #print("Calculated Oscillator 1with2 Resonant Frequency: " + str(calc_res_freq_1with2))
    #print("Actual Oscillator 2 Resonant Frequency: " + str(res_freq2))
    #print("Calculated Oscillator 2with1 Resonant Frequency: " + str(calc_res_freq_2with1))
    #print("Calculated Oscillator 2with2 Resonant Frequency: " + str(calc_res_freq_2with2))
    #print("Oscillator 1 Null Vector 1 Resonant Frequency: " + str(approx_res_freq(k1_1, m1_1, b1_1)))
    #print("Oscillator 1 Null Vector 2 Resonant Frequency: " + str(approx_res_freq(k1_2, m1_2, b1_2)))
    #print("Oscillator 2 Null Vector 1 Resonant Frequency: " + str(approx_res_freq(k2_1, m2_1, b2_1)))
    #print("Oscillator 2 Null Vector 2 Resonant Frequency: " + str(approx_res_freq(k2_2, m2_2, b2_2)))
    
    #print("Weight Ratio from Oscillator 1: " + str(osc1_R))
    #print("Weight Ratio from Oscillator 2: " + str(osc2_R))
    # The R from oscillator 1 seems to work much better, perhaps because it's the one being driven
    R = osc1_R

    # To find the overall weight, we just use the 1D case assuming we know the force
    vect1 = vh[-1]
    vect2 = vh[-2]
    elements = [vect1[k] + R*vect2[k] for k in range(len(vect1))]
    return normalize_elements_1d_by_force(elements, F_set) # does not return the two coefficients


""" mass 1 and mass 2 normalization, 2D nullspace assumption """
# not great for monomer
def normalize_elements_to_m1_m2_assuming_2d(vh, verbose = False, m1_set = m1_set, m2_set = m2_set):
    # elements vector: 'm1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force'
    vect1 = vh[-1]
    vect2 = vh[-2]
    
    if verbose:
        print("If the null-space is 2D, we must be able to independently determine two elements; say it's m1 and m2.")

    # find linear combination such that:
    # a * vect1[0] + b * vect2[0] = m1_set   and
    # a * vect1[1] + b * vect2[1] = m2_set
    ## But this rearranges to:

    coefa = ( vect2[1] * m1_set - m2_set * vect2[0] ) / (vect2[1]*vect1[0] - vect1[1]*vect2[0] )
    coefb = (vect1[1]*m1_set - m2_set *vect1[0] ) /(vect1[1]*vect2[0] - vect2[1]*vect1[0] )

    elements = [coefa*vect1[k]+coefb*vect2[k]  for k in range(len(vect1)) ]
    return elements, coefa, coefb

def normalize_elements_to_m1_set_k1_set_assuming_2d(vh, verbose = False, m1_set = m1_set, k1_set = k1_set):
    # elements vector: 'm1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force'
    vect1 = vh[-1]
    vect2 = vh[-2]
    
    if verbose:
        print("If the null-space is 2D, we must be able to independently determine two elements; say it's m1 and k1.")

    # find linear combination such that:
    # a * vect1[0] + b * vect2[0] = m1_set   and
    # a * vect1[4] + b * vect2[4] = m1_set
    ## But this rearranges to:

    coefa = ( vect2[4] * m1_set - k1_set * vect2[0] ) / (vect2[4]*vect1[0] - vect1[4]*vect2[0] )
    coefb = ( vect1[4] * m1_set - k1_set * vect1[0] ) / (vect1[4]*vect2[0] - vect2[4]*vect1[0] )
    
    if verbose:
        print(str(coefa) + ' of last singular vector and ' + str(coefb) + ' of second to last singular vector.')

    elements = [coefa*vect1[k]+coefb*vect2[k]  for k in range(len(vect1)) ]
    return elements, coefa, coefb

def normalize_elements_to_m1_F_set_assuming_2d(vh, verbose = False, m1_set = m1_set, F_set = F_set):
    # elements vector: 'm1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force'
    vect1 = vh[-1]
    vect2 = vh[-2]
    
    if verbose:
        print("If the null-space is 2D, we must be able to independently determine two elements; say it's m1 and driving force.")


    # find linear combination such that:
    # a * vect1[0] + b * vect2[0] = m1_set   and
    # a * vect1[7] + b * vect2[7] = F_set
    ## But this rearranges to: 

    coefa = ( vect2[7] * m1_set - F_set * vect2[0] ) / (vect2[7]*vect1[0] - vect1[7]*vect2[0] )
    coefb = ( vect1[7] * m1_set - F_set * vect1[0] ) / (vect1[7]*vect2[0] - vect2[7]*vect1[0] )
    
    if verbose:
        print(str(coefa) + ' of last singular vector and ' + str(coefb) + ' of second to last singular vector.')

    elements = [coefa*vect1[k]+coefb*vect2[k]  for k in range(len(vect1)) ]
    return elements, coefa, coefb

# THIS DOESN'T WORK AT ALL; it just selects the 1d nullspace from the 2d nullspace.
def normalize_elements_to_m1_k1_from_1d_assuming_2d(vh, verbose = False, elementsdf1d = elementsdf1d):
    # elements vector: 'm1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force'
    vect1 = vh[-1]
    vect2 = vh[-2]

    # We estimate m1 and k1 from the 1d nullspace and ratchet ourselves more information (but it doesn't work)
    m1_known = elementsdf1d.m1.from_svd 
    k1_known = elementsdf1d.k1.from_svd 

    # find linear combination such that:
    # a * vect1[0] + b * vect2[0] = m1_known   and
    # a * vect1[4] + b * vect2[4] = k1_known
    ## But this rearranges to:

    coefa = ( vect2[4] * m1_known - k1_known * vect2[0] ) / (vect2[4]*vect1[0] - vect1[4]*vect2[0] )
    coefb = (vect1[4]*m1_known - k1_known *vect1[0] ) /(vect1[4]*vect2[0] - vect2[4]*vect1[0] )
    if verbose:
        print('Is the second number zero?? Then you are not 2d')
        print (coefa)
        print (coefb)

    elements = [coefa*vect1[k]+coefb*vect2[k]  for k in range(len(vect1)) ]
    return elements, coefa, coefb

""" Force, mass 1 and mass 2 normalization, 3D nullspace assumption
    Numerics corresponding to m1, m2, F in the elements vector:
    known1 = 0
    known2 = 1
    known3 = 7"""
def normalize_elements_assuming_3d(vh, vals_set = vals_set, known1 = 0, known2 = 1, known3 = 7, verbose = False):
   
    if MONOMER:
        [m1_set,  b1_set,  k1_set,  F_set] = vals_set
    else:
        [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set] = vals_set
    
    # elements vector: 'm1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force'
    vect1 = vh[-1]
    #[m1_1, m2_1, b1_1, b2_1, k1_1, k2_1, c12_1, F_1] = vect1
    vect2 = vh[-2]
    #[m1_2, m2_2, b1_2, b2_2, k1_2, k2_2, c12_2, F_2] = vect2
    vect3 = vh[-3]
    
    if verbose:
        print("If the null-space is 3D, we must be able to independently determine two elements; say it's m1, m2, and F.")

    
    # find linear combination such that:
    # a * vect1[0] + b * vect2[0] + c * vect3[0] = m1_set   and
    # a * vect1[1] + b * vect2[1] + c * vect3[1] = m2_set   and
    # a * vect1[7] + b * vect2[7] + c * vect3[7] = F_set
    ## But this rearranges to:
    
    denom = (vect3[known1] * vect2[known2] * vect1[known3]  - 
             vect2[known1] * vect3[known2] * vect1[known3]  - 
             vect3[known1] * vect1[known2] * vect2[known3]  + 
             vect1[known1] * vect3[known2] * vect2[known3]  + 
             vect2[known1] * vect1[known2] * vect3[known3]  - 
             vect1[known1] * vect2[known2] * vect3[known3])

    coefa = -(-vals_set[known3] * vect3[known1] * vect2[known2]  + 
              vals_set[known3] * vect2[known1] * vect3[known2]  - 
              vals_set[known1] * vect3[known2] * vect2[known3]  + 
              vect3[known1] * vals_set[known2] * vect2[known3]  + 
              vals_set[known1] * vect2[known2] * vect3[known3]  - 
              vect2[known1] * vals_set[known2] * vect3[known3])/ denom
    coefb = -(vals_set[known3] * vect3[known1] * vect1[known2]  - 
              vals_set[known3] * vect1[known1] * vect3[known2]  + 
              vals_set[known1] * vect3[known2] * vect1[known3]  - 
              vect3[known1] * vals_set[known2] * vect1[known3]  - 
              vals_set[known1] * vect1[known2] * vect3[known3]  + 
              vect1[known1] * vals_set[known2] * vect3[known3])/ denom
    coefc = -(-vals_set[known3] * vect2[known1] * vect1[known2]  + 
              vals_set[known3] * vect1[known1] * vect2[known2]  - 
              vals_set[known1] * vect2[known2] * vect1[known3]  + 
              vect2[known1] * vals_set[known2] * vect1[known3]  + 
              vals_set[known1] * vect1[known2] * vect2[known3]  - 
              vect1[known1] * vals_set[known2] * vect2[known3])/ denom

    elements = [coefa*vect1[k]+coefb*vect2[k]+coefc*vect3[k]  for k in range(len(vect1)) ]
    return elements, coefa, coefb, coefc


[M1, M2, B1, B2, K1, K2, K12, FD], coefa, coefb = normalize_elements_to_m1_F_set_assuming_2d(vh)
if MONOMER:
    elements = [M1,  B1,  K1,  FD]
else:
    elements = [M1, M2, B1, B2, K1, K2, K12, FD]
#elements = [element.real for element in elements if element.imag == 0 ]
syserrors = [syserror(elements[i], vals_set[i]) for i in range(len(elements))]
avg2d, rms2d, max2d = combinedsyserror(syserrors, 2)

vals3 = elementlist.copy() # initialize to [['m1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force']]
vals3.append(elements)
vals3.append(vals_set)
vals3.append(syserrors)

elementsdf3 = pd.DataFrame(vals3, index = ['labels', 'from_svd', 'set', "syserror"])
new_header3 = elementsdf3.iloc[0] 
elementsdf3.columns = new_header3
print('1D nullspace results:')
display(elementsdf1d)
print("How good are the 2D nullspace results?")
display(elementsdf3)

#Calculate Rsqrd compared to 2D nullspace to test the 2D nullspace model

plotting = False

amp1_rsqrd2D = rsqrd(model=curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), data=R1_amp, plot=plotting, x=drive)
phase1_rsqrd2D = rsqrd(data=R1_phase, model=theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
amp2_rsqrd2D = rsqrd(data=R2_amp, model=curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
phase2_rsqrd2D = rsqrd(data = R2_phase, model= theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)

print('Noise level: ' + str(noiselevel))
print('R1 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R1.min(), dig=3) + ' to ' 
      + '{num:.{dig}g}'.format(num=df.SNR_R1.max(), dig=3))
if not MONOMER:
    print('R2 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R2.min(), dig=3) + ' to ' 
          + '{num:.{dig}g}'.format(num=df.SNR_R2.max(), dig=3))
print('Average systematic error: ', '{num:.{dig}f}'.format(num=avg2d, dig=3),
      'whereas for 1D it was: ', '{num:.{dig}f}'.format(num=avg1d, dig=3))
print('RMS systematic error:   ', '{num:.{dig}f}'.format(num=rms2d, dig=3), 
      ' whereas for 1D it was: ', '{num:.{dig}f}'.format(num=rms1d, dig=3))
print('Max systematic error:   ', '{num:.{dig}f}'.format(num=max2d, dig=3), 
      ' whereas for 1D it was: ', '{num:.{dig}f}'.format(num=max1d, dig=3))

fig,ax = plt.subplots()
if labelbarchart:
    bargraph = ax.bar(elementsdf3.columns,elementsdf3.transpose()['syserror']*100)
    plt.gca().bar_label(bargraph)
else:
    (elementsdf3.transpose()['syserror']*100).plot(kind='bar')
plt.gca().set_ylabel('systematic error (%)\n')
plt.gca().set_xlabel('');
plt.title("2D nullspace")
plt.show()

print("Number of frequencies: ", len(p))
print("R1 Amp Rsqrd = ", amp1_rsqrd2D,
     "\nR1 Phase Rsqrd = ", phase1_rsqrd2D)
if not MONOMER:
      print("R2 Amp Rsqrd = ", amp2_rsqrd2D,
          "\nR2 Phase Rsqrd = ", phase2_rsqrd2D)


In [ ]:
#Plots of singular value decomposition


"""
measurementdf has a row for each measured frequency
columns: drive, R1Amp, R1Phase, R2Amp, R2Phase, R1AmpCom, R2AmpCom
"""
def plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase, measurementdf,  K1, K2, K12, B1, B2, FD, M1, M2, vals_set=vals_set):
    if MONOMER:
        k12_set = 0 # overwrite the coupling
        [m1_set,  b1_set,  k1_set,  F_set] = vals_set
    else:
        [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set] = vals_set
        
    # This is an approximation for weak coupling, 
    # not so good for strong coupling, but still a nice pair of frequencies to identify.
    res1 = approx_res_freq(k1_set, m1_set, b1_set)
    res2 = approx_res_freq(k2_set, m2_set, b2_set)
        
    Z1 = complexamp(R1_amp, R1_phase)
    Z2 = complexamp(R2_amp, R2_phase)
    
    morefrequencies = np.linspace(min(drive), max(drive), num = len(drive)*10)
    
    R1_amp_noiseless = curve1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
    R1_phase_noiseless = theta1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
    R2_amp_noiseless = curve2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
    R2_phase_noiseless = theta2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)

    R1_real_amp_noiseless = realamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
    R1_im_amp_noiseless = imamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
    R2_real_amp_noiseless = realamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
    R2_im_amp_noiseless = imamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
    
    #fig, ((ax1, ax3),(ax2,ax4),(ax5, ax6)) = plt.subplots(3,2, figsize = (10,10))
    fig, ((ax1, ax3),(ax2,ax4)) = plt.subplots(2,2, figsize = (9.5,5), gridspec_kw={'hspace': 0}, sharex = 'all' )
    
    ax1.plot(morefrequencies, R1_amp_noiseless, '-', color = 'gray', label='set values') # intended curves
    ax1.plot(drive, R1_amp, '.', color = 'lightsteelblue', label='simulated data') # simulated data
    ax1.plot(morefrequencies, curve1(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black', label='SVD results') # predicted spectrum from SVD)
    ax1.set_ylabel('Amplitude\n')
    ax1.set_title('Simulated R1 Spectrum')

    #For loop to plot R1 amplitude values from table
    for i in range(measurementdf.shape[0]):
        ax1.plot(measurementdf.drive[i], measurementdf.R1Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)

    ax2.plot(morefrequencies, R1_phase_noiseless, '-', color = 'gray', label='set values') # intended curves
    ax2.plot(drive, R1_phase, '.', color = 'lightsteelblue')
    ax2.plot(morefrequencies, theta1(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')
    ax2.set_ylabel('Phase (rad)')
    #ax2.set_title('Simulated R1 Phase')

    #For loop to plot R1 amplitude values from table
    for i in range(measurementdf.shape[0]):
        ax2.plot(measurementdf.drive[i], measurementdf.R1Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)

    ax3.plot(morefrequencies, R2_amp_noiseless, '-', color = 'gray', label='set values') # intended curves
    ax3.plot(drive, R2_amp, '.', color = 'lightsteelblue')
    ax3.plot(morefrequencies, curve2(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')
    ax3.set_ylabel('Amplitude\n')
    ax3.set_title('Simulated R2 Spectrum')

    #For loop to plot R1 amplitude values from table
    for i in range(measurementdf.shape[0]):
        ax3.plot(measurementdf.drive[i], measurementdf.R2Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)

    ax4.plot(morefrequencies, R2_phase_noiseless, '-', color = 'gray', label='set values') # intended curves
    ax4.plot(drive, R2_phase, '.', color = 'lightsteelblue')
    ax4.plot(morefrequencies, theta2(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')
    ax4.set_ylabel('Phase (rad)')
    #ax4.set_title('Simulated R2 Phase')

    #For loop to plot R1 amplitude values from table
    for i in range(measurementdf.shape[0]):
        ax4.plot(measurementdf.drive[i], measurementdf.R2Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)

    for ax in [ax1, ax2, ax3, ax4]:
        plt.sca(ax)
        plt.xticks([res1, res2])   
    #for ax in [ax2, ax4]:  
        ax.set_xlabel('Freq (rad/s)')
        
    plt.tight_layout()    

    fig2, ((ax5, ax6)) = plt.subplots(1,2, figsize = (10,4))

    plotcomplex(Z1, drive, 'Complex Amplitude Z1', ax=ax5, label_markers=[res1,res2])
    ax5.scatter(np.real(measurementdf.R1AmpCom), np.imag(measurementdf.R1AmpCom), s=150, facecolors='none', edgecolors='k') 
    plotcomplex(Z2, drive, 'Complex Amplitude Z2', ax=ax6, label_markers=[res1,res2])
    ax6.scatter(np.real(measurementdf.R2AmpCom), np.imag(measurementdf.R2AmpCom), s=150, facecolors='none', edgecolors='k') 

    # true curves
    morefrequencies = np.linspace(0.1, 5, num = n*10)
    ax5.plot(realamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
             imamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
             color='gray', alpha = .5)
    ax6.plot(realamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
             imamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
             color='gray', alpha = .5)

    # svd curves
    morefrequencies = np.linspace(0.1, 5, num = n*10)
    ax5.plot(realamp1(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), 
             imamp1(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')
    ax6.plot(realamp2(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), 
             imamp2(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')

    plt.tight_layout()
    
""" convert from a format used for resultsdf (in the frequency sweep below) 
    to the format used in the above function plot_SVD_results() """
def convert_to_measurementdf(resultsdfitem):
    # columns of measurementdf: drive, R1Amp, R1Phase, R2Amp, R2Phase, R1AmpCom, R2AmpCom
    mdf = pd.DataFrame([[resultsdfitem.Freq1, resultsdfitem.R1Amp_a, resultsdfitem.R1Phase_a, resultsdfitem.R2Amp_a, 
         resultsdfitem.R2Phase_a, resultsdfitem.R1AmpCom_a, resultsdfitem.R2AmpCom_a],
         [resultsdfitem.Freq2, resultsdfitem.R1Amp_b, resultsdfitem.R1Phase_b, resultsdfitem.R2Amp_b, 
         resultsdfitem.R2Phase_b, resultsdfitem.R1AmpCom_b, resultsdfitem.R2AmpCom_b]],
            columns = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase', 'R1AmpCom', 'R2AmpCom'])
    return mdf
    
print("2D nullspace")
plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase,df,  K1, K2, K12, B1, B2, FD, M1, M2)

In [ ]:
"""## What if it's 3D nullspace? No, it's not.

[M1, M2, B1, B2, K1, K2, K12, FD], coefa, coefb, coefc = normalize_elements_assuming_3d(vh)
if MONOMER:
    elements = [M1,  B1,  K1,  FD]
else:
    elements = [M1, M2, B1, B2, K1, K2, K12, FD]
#elements = [element.real for element in elements if element.imag == 0 ]
syserrors = [syserror(elements[i], vals_set[i]) for i in range(len(elements))]
avg3d, rms3d, max3d = combinedsyserror(syserrors, 3)

vals3d = elementlist.copy() # initialize to [['m1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force']]
vals3d.append(elements)
vals3d.append(vals_set)
vals3d.append(syserrors)

elementsdf3d = pd.DataFrame(vals3d, index = ['labels', 'from_svd', 'set', "syserror"])
new_header3d = elementsdf3d.iloc[0] 
elementsdf3d.columns = new_header3d
print('1D nullspace results:')
display(elementsdf1d)
print("2D nullspace results:")
display(elementsdf3)
print("3D nullspace results:")
display(elementsdf3d)

#Calculate Rsqrd compared to 3D nullspace to test the 3D nullspace model

plotting = False

amp1_rsqrd3D = rsqrd(model=curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), data=R1_amp, plot=plotting, x=drive)
phase1_rsqrd3D = rsqrd(data=R1_phase, model=theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
amp2_rsqrd3D = rsqrd(data=R2_amp, model=curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
phase2_rsqrd3D = rsqrd(data = R2_phase, model= theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)

print('Noise level: ' + str(noiselevel))
print('R1 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R1.min(), dig=3) + ' to ' 
      + '{num:.{dig}g}'.format(num=df.SNR_R1.max(), dig=3))
if not MONOMER:
    print('R2 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R2.min(), dig=3) + ' to ' 
          + '{num:.{dig}g}'.format(num=df.SNR_R2.max(), dig=3))
print('Average 3D systematic error: ', '{num:.{dig}f}'.format(num=avg3d, dig=3),
      'whereas for 1D it was: ', '{num:.{dig}f}'.format(num=avg1d, dig=3))
print('RMS 3D systematic error:   ', '{num:.{dig}f}'.format(num=rms3d, dig=3), 
      ' whereas for 1D it was: ', '{num:.{dig}f}'.format(num=rms1d, dig=3))
print('Max 3D systematic error:   ', '{num:.{dig}f}'.format(num=max3d, dig=3), 
      ' whereas for 1D it was: ', '{num:.{dig}f}'.format(num=max1d, dig=3))

fig,ax = plt.subplots()
if labelbarchart:
    bargraph = ax.bar(elementsdf3.columns,elementsdf3d.transpose()['syserror']*100)
    plt.gca().bar_label(bargraph)
else:
    (elementsdf3d.transpose()['syserror']*100).plot(kind='bar')
plt.gca().set_ylabel('systematic error (%)\n')
plt.gca().set_xlabel('');
plt.title("3D nullspace")
plt.show()

print("Number of frequencies: ", len(p))
print("R1 Amp Rsqrd = ", amp1_rsqrd3D,
     "\nR1 Phase Rsqrd = ", phase1_rsqrd3D)
if not MONOMER:
      print("R2 Amp Rsqrd = ", amp2_rsqrd3D,
          "\nR2 Phase Rsqrd = ", phase2_rsqrd3D)
        
print("3D nullspace")
plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase,df,  K1, K2, K12, B1, B2, FD, M1, M2)"""

In [ ]:
stophere

In [ ]:
# varying coupling strength k_12 

# desiredfreqs = [2.1, 2.4]
#desiredfreqs = [res1,res2]
desiredfreqs = [res1,res2, 2.1, 2.4]

p = freqpoints(desiredfreqs, drive)

#p = [14,15,16,29,30,31] #indices of frequencies to use
#p=[14,16]
p = range(len(drive)) # use all the frequencies that are available
print(p)

"""
try:
    del k12_set # just to make sure there's no mistake
except NameError:
    print('re-running')
"""

# options
verbose = False
numk12 = 100
move_peaks = False

#initialize
if MONOMER:
    k12_list = [0]
else:
    k12_list = np.linspace(0.1, 60, num = numk12);
measurementdflist = []
results = []
SNRtable = []
first = True

for this_k12_set in k12_list:
    
    if not MONOMER:
        vals_set_vary = [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, this_k12_set, F_set]

    ## calculate the spectra with this_k12_set
    R1_amp, R1_phase, R2_amp, R2_phase, R1_real_amp, R1_im_amp, R2_real_amp, R2_im_amp = \
        calculate_spectra(drive, k1_set, k2_set, this_k12_set, b1_set, b2_set, F_set, m1_set, m2_set, n)
    
    ## find peaks and choose frequency locations that match
    if move_peaks:
        newp, heights = find_peaks(R2_amp, height=.015)

        if len(newp) < 1: # just use the default if you didn't find a peak
            newp = p
            print("Didn't find a peak")
        elif len(newp) < 2: # only found one peak; use the default for the other peak.
            if newp[0] != p[1]:
                p[0] = newp[0]
                print('Found one peak,' + str(newp))
            else:
                print('Found one peak, ' + str(newp)  + ' but it is the same as what I would have used anyway.')
        if len(newp) > 2: 
            print('Found too many peaks, ' + str(newp) + str(heights))
            if newp[0] <=2:
                newp = newp[1:] # remove that 
            p_df = pd.DataFrame.join(pd.DataFrame(data = newp), pd.DataFrame(data=heights))
            while len(p_df) > 2:
                minindex = p_df[['peak_heights']].idxmin()
                p_df = p_df.drop(p_df.index[minindex])

            p = p_df[[0]].transpose().values.tolist()[0]

        print(p)

    SNR_R1_f1,SNR_R2_f1, A1f1avg, A1f1std, A2f1avg, A2f1std = SNRcalc(
        drive[p[0]], vals_set=vals_set_vary, noiselevel = noiselevel, detailed = True)
    SNR_R1_f2,SNR_R2_f2, A1f2avg, A1f2std, A2f2avg, A2f2std = SNRcalc(
        drive[p[1]], vals_set=vals_set_vary, noiselevel = noiselevel, detailed = True)
    
    if first:
        fig, SNRax = plt.subplots()
    thistable = []
    for i in range(len(p)):
        SNR1, SNR2 = SNRcalc(drive[p[i]], vals_set=vals_set_vary, noiselevel = noiselevel, plot = True, ax = SNRax,)
        thistable.append([this_k12_set, drive[p[i]], R1_amp[p[i]], R1_phase[p[i]], R2_amp[p[i]], R2_phase[p[i]], 
                      complexamp(R1_amp[p[i]],R1_phase[p[i]] ),
                      complexamp(R2_amp[p[i]], R2_phase[p[i]]),
                      SNR1, SNR2,
                     syserror(R1_amp[p[i]], R1_amp_noiseless[p[i]]),
                     (R1_phase[p[i]] - R1_phase_noiseless[p[i]]),
                     syserror(R2_amp[p[i]],R2_amp_noiseless[p[i]]),
                     (R2_phase[p[i]]-R2_phase_noiseless[p[i]]),
                     ])

    thismeasurementdf = pd.DataFrame(data = thistable, columns = ['k12', 'drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase',
                                               'R1AmpCom', 'R2AmpCom',
                                               'SNR_R1','SNR_R2',
                                               'R1Amp_syserror', 'R1Phase_diff', 'R2Amp_syserror', 'R2Phase_diff'])
    if verbose:
        display(thismeasurementdf)
    measurementdflist.append(thismeasurementdf)
    
    Zmatrix = Zmatrix2resonators(thismeasurementdf, 
                                     frequencycolumn = 'drive', 
                                     complexamplitude1 = 'R1AmpCom', 
                                     complexamplitude2 = 'R2AmpCom',
                                     dtype = np.double) # storing it as complex avoids a runtime warning
        
    #display(pd.DataFrame(Zmatrix))

    #SVD,  u and vh are 2D unitary arrays and s is a 1D array of the input matrix's singular values
    u, s, vh = np.linalg.svd(Zmatrix, full_matrices = True)

    ## 1D NULLSPACE
    # extract parameters found by SVD
    #assign variables # mass of resonator 1, mass 2, damping 1, damping 2, stiffness 1, stiffness 2, coupling stiffness, force
    [M1, M2, B1, B2, K1, K2, K12, FD] = vh[-1] # the 7th singular value is the smallest one (closest to zero)

    # normalize elements vector to the force, assuming 1D nullspace
    allelements = normalize_elements_1d_by_force([M1, M2, B1, B2, K1, K2, K12, FD], F_set)
    # recast as real, not complex # but real gets a warning
    # allelements = [thiselement.real for thiselement in allelements if thiselement.imag == 0 ]
    M1, M2, B1, B2, K1, K2, K12, FD = allelements
        
    # Alternative check: 
    # calculate how close the SVD-determined parameters are compared to the originally set parameters

    if MONOMER:
        el = [M1,  B1,  K1,  FD]
    else:
        el = [M1, M2, B1, B2, K1, K2, K12, FD]
        
    syserrors = [syserror(el[i], vals_set_vary[i]) for i in range(len(el))]

    # Values to compare:
    # Set values: k1_set, k2_set, this_k12_set, b1_set, b2_set, F_set, m1_set, m2_set
    # SVD-determined values: M1, M2, B1, B2, K1, K2, K12, FD
    K1syserror = syserror(K1,k1_set)
    K2syserror = syserror(K2,k2_set)
    K12syserror = syserror(K12,this_k12_set)
    B1syserror = syserror(B1,b1_set)
    B2syserror = syserror(B2,b2_set)
    FDsyserror = syserror(FD,F_set)
    M1syserror = syserror(M1,m1_set)
    M2syserror = syserror(M2,m2_set)
    avgsyserror, rmssyserror, maxsyserr = combinedsyserror(syserrors,1) # subtract 1 degrees of freedom for 1D nullspace
    """        parametererror2 = np.sqrt((K1syserror**2 + K2syserror**2 + K12syserror**2 \
                              + B1syserror**2 + B2syserror**2 + FDsyserror**2 + \
                              M1syserror**2 + M2syserror**2)/8)"""

    ### Normalize elements in 2D nullspace 
    """ # Problem: res1 formula only for weak coupling.
    [M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D] = \
        normalize_elements_to_res1_and_F_2d(vh, vals_set = vals_set_vary)
    coefa = np.nan
    coefb = np.nan"""
    #[M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D], coefa, coefb = \
    #    normalize_elements_to_m1_set_k1_set_assuming_2d(vh)
    #[M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D], coefa, coefb = \
    #    normalize_elements_to_m1_m2_assuming_2d(vh, verbose = False, m1_set = m1_set, m2_set = m2_set)
    [M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D], coefa, coefb = \
        normalize_elements_to_m1_F_set_assuming_2d(vh, verbose = False, m1_set = m1_set, F_set = F_set)
    normalizationpair = 'm1 and F'
    if MONOMER:
        el_2D = M1_2D, B1_2D, K1_2D,  FD_2D
    else:
        el_2D = M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D 
    syserrors_2D = [syserror(el_2D[i], vals_set_vary[i]) for i in range(len(el_2D))]

    # Values to compare:
    # Set values: k1_set, k2_set, this_k12_set, b1_set, b2_set, F_set, m1_set, m2_set
    # SVD-determined values: M1, M2, B1, B2, K1, K2, K12, FD

    K1syserror_2D = syserror(K1_2D,k1_set)
    B1syserror_2D = syserror(B1_2D,b1_set)
    FDsyserror_2D = syserror(FD_2D,F_set)
    M1syserror_2D = syserror(M1_2D,m1_set)

    if MONOMER:
        K2syserror_2D = 0
        K12syserror_2D = 0
        B2syserror_2D = 0
        M2syserror_2D = 0
    else:
        K2syserror_2D = syserror(K2_2D,k2_set)
        K12syserror_2D = syserror(K12_2D,this_k12_set)
        B2syserror_2D = syserror(B2_2D,b2_set)
        M2syserror_2D = syserror(M2_2D,m2_set)

    avgsyserror_2D, rmssyserror_2D, maxsyserr_2D = combinedsyserror(syserrors_2D,2) # subtract 2 degrees of freedom for 2D nullspace

    if first:
        plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase, thismeasurementdf,  K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, vals_set_vary)
        plt.show()
        first = False
    
    # save the results (note: I mean saving to temporary kernel memory, not to harddrive)
    results.append([this_k12_set,
                    drive[p[0]],
                      drive[p[1]],
                      drive[p[1]] - drive[p[0]],
                      R1_amp[p[0]], R1_phase[p[0]], R2_amp[p[0]], R2_phase[p[0]],
                              R1_amp[p[1]], R1_phase[p[1]], R2_amp[p[1]], R2_phase[p[1]],
                      M1, M2, B1, B2, K1, K2, K12, FD, # elements vector, from 1D SVD
                      K1syserror,
                      K2syserror,
                      K12syserror,
                      B1syserror,
                      B2syserror,
                      FDsyserror,
                      M1syserror,
                      M2syserror,
                      avgsyserror, rmssyserror,maxsyserr,
                     np.sum(R1_amp - curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2,   # sum of squares
                    np.sum((R1_phase - theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                    np.sum((R2_amp - curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                    np.sum((R2_phase - theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                      M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D, # elements vector, from 2D SVD
                      K1syserror_2D,
                      K2syserror_2D,
                      K12syserror_2D,
                      B1syserror_2D,
                      B2syserror_2D,
                      FDsyserror_2D,
                      M1syserror_2D,
                      M2syserror_2D,
                      avgsyserror_2D, rmssyserror_2D,maxsyserr_2D,
                    # sum of squares
                     np.sum(R1_amp - curve1(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2, 
                    np.sum((R1_phase - theta1(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                    np.sum((R2_amp - curve2(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                    np.sum((R2_phase - theta2(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                    avgsyserror - avgsyserror_2D,
                    s[-1],s[-2], # smallest two singular values
                    SNR_R1_f1,SNR_R2_f1, # signal to noise ratio
                    A1f1avg, A1f1std, A2f1avg, A2f1std,
                    SNR_R1_f2,SNR_R2_f2,
                     A1f2avg, A1f2std, A2f2avg, A2f2std,
                   R1_amp_noiseless[p[0]], R1_phase_noiseless[p[0]], R2_amp_noiseless[p[0]], R2_phase_noiseless[p[0]],
                   R1_amp_noiseless[p[1]], R1_phase_noiseless[p[1]], R2_amp_noiseless[p[1]], R2_phase_noiseless[p[1]],
                   R1_phase_noiseless[p[1]]-R1_phase_noiseless[p[0]]])
        

resultsdfk12 = pd.DataFrame(
    data=results, 
    columns = ['k12_set' , 'Freq1','Freq2', 'Difference',
               'R1_amp_meas_a', 'R1_phase_meas_a', 'R2_amp_meas_a', 'R2_phase_meas_a',
               'R1_amp_meas_b', 'R1_phase_meas_b', 'R2_amp_meas_b', 'R2_phase_meas_b',
        'M1', 'M2', 'B1', 'B2', 'K1', 'K2', 'K12', 'FD', # elements vector, from 1D SVD
        'K1syserror',
        'K2syserror',
        'K12syserror',
        'B1syserror',
        'B2syserror',
        'FDsyserror',
        'M1syserror',
        'M2syserror',
        'avgsyserror', 'rmssyserror', 'maxsyserr',
        'R1AmpFit', 'R1PhaseFit', 'R2AmpFit', 'R2PhaseFit',   # sum of squares
        'M1_2D', 'M2_2D', 'B1_2D', 'B2_2D', 'K1_2D', 'K2_2D', 'K12_2D', 'FD_2D', # elements vector, from 2D SVD
        'K1syserror_2D',
        'K2syserror_2D',
        'K12syserror_2D',
        'B1syserror_2D',
        'B2syserror_2D',
        'FDsyserror_2D',
        'M1syserror_2D',
        'M2syserror_2D',
        'avgsyserror_2D', 'rmssyserror_2D', 'maxsyserr_2D',
        'R1AmpFit_2D', 'R1PhaseFit_2D', 'R2AmpFit_2D', 'R2PhaseFit_2D', # sum of squares
        'avgsyserror-avgsyserror_2D',
        'smallest singular value', 'second smallest singular value',
        'SNR_R1_f1','SNR_R2_f1',
               'A1f1avg', 'A1f1std', 'A2f1avg', 'A2f1std',
        'SNR_R1_f2','SNR_R2_f2',
                'A1f2avg', 'A1f2std', 'A2f2avg', 'A2f2std',
        'R1_amp_noiseless_a', 'R1_phase_noiseless_a', 'R2_amp_noiseless_a', 'R2_phase_noiseless_a',
        'R1_amp_noiseless_b', 'R1_phase_noiseless_b', 'R2_amp_noiseless_b', 'R2_phase_noiseless_b', 'R1_phase_diff'])

resultsdfk12

In [ ]:
maxsyserror_to_plot = 10

""" count how many times each of the following         'K1syserror_2D',
        'K2syserror_2D',
        'K12syserror_2D',
        'B1syserror_2D',
        'B2syserror_2D',
        'FDsyserror_2D',
        'M1syserror_2D',
        'M2syserror_2D', is the worst """
plt.figure()
plt.plot(resultsdfk12.k12_set, resultsdfk12.B2syserror_2D*100, label='B2')
plt.plot(resultsdfk12.k12_set, resultsdfk12.M2syserror_2D*100, label='M2')

plt.plot(resultsdfk12.k12_set, resultsdfk12.K2syserror_2D *100, label='K2')
plt.plot(resultsdfk12.k12_set, resultsdfk12.K12syserror_2D*100, label='K12')
plt.plot(resultsdfk12.k12_set, resultsdfk12.B1syserror_2D*100, label='B1')
plt.plot(resultsdfk12.k12_set, resultsdfk12.K1syserror_2D*100, label='K1')
plt.plot(resultsdfk12.k12_set, resultsdfk12.M1syserror_2D*100, label='M1')
plt.plot(resultsdfk12.k12_set, resultsdfk12.FDsyserror_2D*100, label='F')
plt.title('2D nullspace normalized by ' + normalizationpair)
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1.05), ncol=1,)
plt.ylim(0,25)
plt.ylabel('Syserr (%)');
plt.xlabel('$k_{12, set}$ (arb. units)');

plt.figure()
plt.plot(resultsdfk12.k12_set, 100*resultsdfk12.avgsyserror_2D)
#plt.plot(resultsdfk12.k12_set, 100*resultsdfk12.avgsyserror)
#plt.plot(resultsdfk12.k12_set, 100*resultsdfk12.rmssyserror_2D)

plt.title('2D nullspace normalized by ' + normalizationpair)
plt.xlabel('$k_{12, set}$ (arb. units)')
plt.ylabel('Average syserror (%)');
plt.ylim(0,ymax=maxsyserror_to_plot)

plt.figure()
plt.plot(resultsdfk12.k12_set,resultsdfk12.SNR_R1_f1, label="R1, f1" )
plt.plot(resultsdfk12.k12_set,resultsdfk12.SNR_R1_f2, label="R1, f2" )
plt.plot(resultsdfk12.k12_set,resultsdfk12.SNR_R2_f1, label="R2, f1" )
plt.plot(resultsdfk12.k12_set,resultsdfk12.SNR_R2_f2, label="R2, f2" )
plt.ylim(ymin=0)
plt.legend()
plt.xlabel('$k_{12, set}$ (arb. units)')
plt.ylabel('SNR');

plt.figure()
plt.plot(resultsdfk12.k12_set,resultsdfk12.R1_amp_meas_a, label="R1, f1" )
plt.plot(resultsdfk12.k12_set,resultsdfk12.R1_amp_meas_b, label="R1, f2")
plt.plot(resultsdfk12.k12_set,resultsdfk12.R2_amp_meas_a, label="R2, f1")
plt.plot(resultsdfk12.k12_set,resultsdfk12.R2_amp_meas_b, label="R2, f2")
plt.legend()
plt.xlabel('$k_{12, set}$ (arb. units)')
plt.ylabel('amplitude (arb. units)');

"""
# Make sure that SNRcalc is being used correctly.
plt.figure()
plt.plot(resultsdfk12.k12_set,resultsdfk12.R1_amp_meas_a, label="R1, f1, measured" )
plt.plot(resultsdfk12.k12_set,resultsdfk12.A1f1avg, label='R1, f1,average measured')
plt.legend()
plt.xlabel('$k_{12, set}$ (arb. units)')
plt.ylabel('amplitude (arb. units)');
"""

plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase, thismeasurementdf,  
                 K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 
                 vals_set_vary)

plt.figure()
if numk12>400:
    alpha = .4
else:
    alpha = .8
if MONOMER:
    sc = plt.scatter(resultsdfk12.SNR_R1_f2, resultsdfk12.SNR_R1_f1, s=10, c = resultsdfk12.rmssyserror_2D *100, cmap = 'magma_r', vmax = maxsyserror_to_plot , alpha = alpha) 
    plt.xlabel('SNR_R1_f2')
    plt.ylabel('SNR_R1_f1')
else:
    sc = plt.scatter(resultsdfk12.SNR_R2_f2, resultsdfk12.SNR_R2_f1, s=10, c = resultsdfk12.rmssyserror_2D *100, cmap = 'magma_r', vmax = maxsyserror_to_plot , alpha = alpha) 
    plt.xlabel('SNR_R2_f2')
    plt.ylabel('SNR_R2_f1')
cbar = plt.colorbar(sc)
cbar.outline.set_visible(False)
cbar.set_label('rmssyserror_2D (%)')
plt.gca().axis('equal');

plt.figure()
if MONOMER:
    sc = plt.scatter(resultsdfk12.SNR_R1_f2, resultsdfk12.SNR_R1_f1, s=10, c = resultsdfk12.k12_set , cmap = 'viridis', alpha = alpha) 
    plt.xlabel('SNR_R1_f2')
    plt.ylabel('SNR_R1_f1')
else:
    sc = plt.scatter(resultsdfk12.SNR_R2_f2, resultsdfk12.SNR_R2_f1, s=10, c = resultsdfk12.k12_set , cmap = 'viridis', alpha = alpha) 
    plt.xlabel('SNR_R2_f2')
    plt.ylabel('SNR_R2_f1')
cbar = plt.colorbar(sc)
cbar.outline.set_visible(False)
cbar.set_label('$k_{12, set}$ (arb. units)')
plt.gca().axis('equal');



In [ ]:
stophere

**Sweep through possible pairs of frequencies**

In [ ]:
#Code that loops through points of different spacing

#initiate arrays for badness of fit values
fits = []

# initate array for results
results = []

SNRtable = []

# Loop over possible combinations of frequency indices, i1 and i2
for i1 in range(n):
    freq1 = drive[i1]
    SNR_R1_f1,SNR_R2_f1 = SNRcalc(freq1)
    
    SNRtable.append([freq1, SNR_R1_f1,SNR_R2_f1])
        
    for i2 in range(n):
        freq2 = drive[i2]
        
        # This might be slow but I can refactor later to not recalculate this on every loop
        SNR_R1_f2,SNR_R2_f2 = SNRcalc(freq2)
        
        ## Recalculate the noise for each pair. (I checked: Python doesn't cache the function returns.)
        # Values for frequency 1
        na11,np11, Z1_1 = noisyR1ampphase(freq1)
        na21,np21, Z2_1 = noisyR2ampphase(freq1)
        na11 = na11[0] # get rid of the arrays; just want raw number
        np11 = np11[0]
        np21 = np21[0]
        na21 = na21[0]
        Z1_1 = Z1_1[0]
        Z2_1 = Z2_1[0]

        
        # Values for frequency 2
        na12,np12, Z1_2 = noisyR1ampphase(freq2)
        na22,np22, Z2_2 = noisyR2ampphase(freq2)
        na12 = na12[0]
        np12 = np12[0]
        na22 = na22[0]
        np22 = np22[0]
        Z1_2 = Z1_2[0]
        Z2_2 = Z2_2[0]

        columnnames = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase', 'R1AmpCom', 'R2AmpCom']
        simulateddata = [[freq1, na11, np11, na21,np21, Z1_1, Z2_1],
                         [freq2, na12,np12, na22,np22, Z1_2, Z2_2]]
        measurementdf = pd.DataFrame(data = simulateddata, columns = columnnames)

        Zmatrix = Zmatrix2resonators(measurementdf, 
                                     frequencycolumn = 'drive', 
                                     complexamplitude1 = 'R1AmpCom', 
                                     complexamplitude2 = 'R2AmpCom',
                                     dtype = np.double) # storing it as complex avoids a runtime warning
        
        #display(pd.DataFrame(Zmatrix))

        #SVD,  u and vh are 2D unitary arrays and s is a 1D array of the input matrix's singular values
        u, s, vh = np.linalg.svd(Zmatrix, full_matrices = True)

        ## 1D NULLSPACE
        # extract parameters found by SVD
        #assign variables # mass of resonator 1, mass 2, damping 1, damping 2, stiffness 1, stiffness 2, coupling stiffness, force
        [M1, M2, B1, B2, K1, K2, K12, FD] = vh[-1] # the 7th singular value is the smallest one (closest to zero)

        # normalize elements vector to the force, assuming 1D nullspace
        allelements = normalize_elements_1d_by_force([M1, M2, B1, B2, K1, K2, K12, FD], F_set)
        # recast as real, not complex # but real gets a warning
        # allelements = [thiselement.real for thiselement in allelements if thiselement.imag == 0 ]
        M1, M2, B1, B2, K1, K2, K12, FD = allelements
        
        # Alternative check: 
        # calculate how close the SVD-determined parameters are compared to the originally set parameters

        if MONOMER:
            el = [M1,  B1,  K1,  FD]
        else:
            el = [M1, M2, B1, B2, K1, K2, K12, FD]
        
        syserrors = [syserror(el[i], vals_set[i]) for i in range(len(el))]
     
        # Values to compare:
        # Set values: k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set
        # SVD-determined values: M1, M2, B1, B2, K1, K2, K12, FD
        K1syserror = syserror(K1,k1_set)
        K2syserror = syserror(K2,k2_set)
        K12syserror = syserror(K12,k12_set)
        B1syserror = syserror(B1,b1_set)
        B2syserror = syserror(B2,b2_set)
        FDsyserror = syserror(FD,F_set)
        M1syserror = syserror(M1,m1_set)
        M2syserror = syserror(M2,m2_set)
        avgsyserror, rmssyserror, maxsyserr = combinedsyserror(syserrors,1) # subtract 1 degrees of freedom for 1D nullspace
        """        parametererror2 = np.sqrt((K1syserror**2 + K2syserror**2 + K12syserror**2 \
                                  + B1syserror**2 + B2syserror**2 + FDsyserror**2 + \
                                  M1syserror**2 + M2syserror**2)/8)"""
        
        [M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D], coefa, coefb = \
            normalize_elements_to_m1_set_k1_set_assuming_2d(vh)
        if MONOMER:
            el_2D = M1_2D, B1_2D, K1_2D,  FD_2D
        else:
            el_2D = M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D 
        syserrors_2D = [syserror(el_2D[i], vals_set[i]) for i in range(len(el_2D))]
     
        # Values to compare:
        # Set values: k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set
        # SVD-determined values: M1, M2, B1, B2, K1, K2, K12, FD

        K1syserror_2D = syserror(K1_2D,k1_set)
        B1syserror_2D = syserror(B1_2D,b1_set)
        FDsyserror_2D = syserror(FD_2D,F_set)
        M1syserror_2D = syserror(M1_2D,m1_set)
        
        if MONOMER:
            K2syserror_2D = 0
            K12syserror_2D = 0
            B2syserror_2D = 0
            M2syserror_2D = 0
        else:
            K2syserror_2D = syserror(K2_2D,k2_set)
            K12syserror_2D = syserror(K12_2D,k12_set)
            B2syserror_2D = syserror(B2_2D,b2_set)
            M2syserror_2D = syserror(M2_2D,m2_set)

        avgsyserror_2D, rmssyserror_2D, maxsyserr_2D = combinedsyserror(syserrors_2D,2) # subtract 2 degrees of freedom for 2D nullspace

        # save the results (note: I mean saving to temporary kernel memory, not to harddrive)
        results.append([drive[i1],
                          drive[i2],
                          drive[i2] - drive[i1],
                        na11, np11, na21,np21, Z1_1, Z2_1, # frequency 1 measurements
                        na12,np12, na22,np22, Z1_2, Z2_2,  # frequency 2 measurements
                          M1, M2, B1, B2, K1, K2, K12, FD, # elements vector, from 1D SVD
                          K1syserror,
                          K2syserror,
                          K12syserror,
                          B1syserror,
                          B2syserror,
                          FDsyserror,
                          M1syserror,
                          M2syserror,
                          avgsyserror, rmssyserror,maxsyserr,
                         np.sum(R1_amp - curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2,   # sum of squares
                        np.sum((R1_phase - theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                        np.sum((R2_amp - curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                        np.sum((R2_phase - theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                          M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D, # elements vector, from 2D SVD
                          K1syserror_2D,
                          K2syserror_2D,
                          K12syserror_2D,
                          B1syserror_2D,
                          B2syserror_2D,
                          FDsyserror_2D,
                          M1syserror_2D,
                          M2syserror_2D,
                          avgsyserror_2D, rmssyserror_2D,maxsyserr_2D,
                        # sum of squares
                         np.sum(R1_amp - curve1(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2, 
                        np.sum((R1_phase - theta1(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                        np.sum((R2_amp - curve2(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                        np.sum((R2_phase - theta2(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                        avgsyserror - avgsyserror_2D,
                        s[-1],s[-2], # smallest two singular values
                        SNR_R1_f1,SNR_R2_f1, # signal to noise ratio
                        SNR_R1_f2,SNR_R2_f2,
                       R1_amp_noiseless[i1], R1_phase_noiseless[i1], R2_amp_noiseless[i1], R2_phase_noiseless[i1],
                       R1_amp_noiseless[i2], R1_phase_noiseless[i2], R2_amp_noiseless[i2], R2_phase_noiseless[i2],
                       R1_phase_noiseless[i2]-R1_phase_noiseless[i1]])
        

resultsdf = pd.DataFrame(
    data=results, 
    columns = ['Freq1','Freq2', 'Difference',
               'R1Amp_a', 'R1Phase_a', 'R2Amp_a', 'R2Phase_a', 'R1AmpCom_a', 'R2AmpCom_a', # frequency 1 measurements
               'R1Amp_b', 'R1Phase_b', 'R2Amp_b', 'R2Phase_b', 'R1AmpCom_b', 'R2AmpCom_b', # frequency 2 measurements
        'M1', 'M2', 'B1', 'B2', 'K1', 'K2', 'K12', 'FD', # elements vector, from 1D SVD
        'K1syserror',
        'K2syserror',
        'K12syserror',
        'B1syserror',
        'B2syserror',
        'FDsyserror',
        'M1syserror',
        'M2syserror',
        'avgsyserror', 'rmssyserror', 'maxsyserr',
        'R1AmpFit', 'R1PhaseFit', 'R2AmpFit', 'R2PhaseFit',   # sum of squares
        'M1_2D', 'M2_2D', 'B1_2D', 'B2_2D', 'K1_2D', 'K2_2D', 'K12_2D', 'FD_2D', # elements vector, from 2D SVD
        'K1syserror_2D',
        'K2syserror_2D',
        'K12syserror_2D',
        'B1syserror_2D',
        'B2syserror_2D',
        'FDsyserror_2D',
        'M1syserror_2D',
        'M2syserror_2D',
        'avgsyserror_2D', 'rmssyserror_2D', 'maxsyserr_2D',
        'R1AmpFit_2D', 'R1PhaseFit_2D', 'R2AmpFit_2D', 'R2PhaseFit_2D', # sum of squares
        'avgsyserror-avgsyserror_2D',
        'smallest singular value', 'second smallest singular value',
        'SNR_R1_f1','SNR_R2_f1',
        'SNR_R1_f2','SNR_R2_f2',
        'R1_amp_noiseless_a', 'R1_phase_noiseless_a', 'R2_amp_noiseless_a', 'R2_phase_noiseless_a',
        'R1_amp_noiseless_b', 'R1_phase_noiseless_b', 'R2_amp_noiseless_b', 'R2_phase_noiseless_b', 'R1_phase_diff'])

In [ ]:
with pd.option_context('display.max_rows', None,):
    display(resultsdf[245:249].transpose())

In [ ]:
resultsdf['avgsyserror-avgsyserror_2D'].min()

In [ ]:
print('The most likely frequency pair to be 1d nullspace:')

#labellists
if MONOMER:
    elemslist = ['M1', 'B1', 'K1', 'FD']
    bardisplaylabels =  ['K1', 'B1','FD','M1','avg', 'rms']
else:
    elemslist = ['M1', 'M2', 'B1', 'B2', 'K1', 'K2', 'K12', 'FD']
    bardisplaylabels =  ['K1', 'K2', 'K12','B1','B2','FD','M1','M2','avg', 'rms']
elemslist_2D = [el + '_2D' for el in elemslist]
llist1 = ['Freq1', 'Freq2', 'avgsyserror-avgsyserror_2D', 'rmssyserror', 'rmssyserror_2D'] +  elemslist + elemslist_2D
syserrorlist = [w + 'syserror' for w in bardisplaylabels]
syserrorlist_2D = [w + '_2D' for w in syserrorlist]

min_df = resultsdf.loc[resultsdf['avgsyserror-avgsyserror_2D'].argmin()] # most likely to be 1d nullspace
display(min_df[llist1])
#min_df[['M1_2D', 'M2_2D', 'B1_2D', 'B2_2D', 'K1_2D', 'K2_2D', 'K12_2D', 'FD_2D',]]

def grapherror_1D_2D(syserrordf, bardisplaylabels, syserrorlist, syserrorlist_2D ):
    X = np.arange(len(bardisplaylabels))
    fig, ax = plt.subplots()
    ax.bar(X + 0.2,syserrordf[syserrorlist], color = 'b', width = 0.3)
    ax.set_xticks(X+ 0.25, bardisplaylabels)
    ax.bar(X + 0.50, syserrordf[syserrorlist_2D], color = 'r', width = 0.3)
    plt.title('syserrors: 1d blue, 2d red');
    
grapherror_1D_2D(min_df, bardisplaylabels, syserrorlist, syserrorlist_2D )
plt.show()

print('1D nullspace')
plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase,convert_to_measurementdf(min_df),  
                 min_df.K1, min_df.K2, min_df.K12, min_df.B1, min_df.B2, min_df.FD, min_df.M1, min_df.M2)
plt.show()
print('The above is likely to be quite a poor choice of frequencies, since it was selected for having poor 2d nullspace results')

In [ ]:
with pd.option_context('display.max_rows', None,):
    display(min_df)

In [ ]:
best_df = resultsdf.loc[resultsdf['avgsyserror_2D'].argmin()] # most likely to be good
display(best_df[llist1])
print('Best 2d results (lowest average syserr):')
grapherror_1D_2D(best_df, bardisplaylabels, syserrorlist, syserrorlist_2D )
plt.show()

print('2D nullspace, best choice of two frequencies')
plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase,convert_to_measurementdf(best_df),  
                 best_df.K1_2D, best_df.K2_2D, best_df.K12_2D, best_df.B1_2D, best_df.B2_2D, 
                 best_df.FD_2D, best_df.M1_2D, best_df.M2_2D)
plt.show()
print('2D nullspace, best choice of two frequencies')

In [ ]:
figsize = (8*3/2,7.7)
sns.set_context('notebook')

fig, ((ax1, ax2, ax7), (ax3, ax4, ax4b), (ax5, ax6, ax6b)) = plt.subplots(3, 3, figsize=figsize)

#print('Assuming 1D nullspace (where is this assumption ok?)')

print('Noiselevel: ' + str(noiselevel))

vmax = .3

plt.sca(ax1)
lambdagrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'smallest singular value').sort_index(axis = 0, ascending = False)
myheatmap(lambdagrid, "smallest singular value", cmap = 'Greys'); 

plt.sca(ax2)
lambda2grid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'second smallest singular value').sort_index(axis = 0, ascending = False)
myheatmap(lambda2grid, "2nd smallest\nsingular value", cmap = 'viridis'); 

plt.sca(ax3)
if MONOMER:
    choice = 'B1syserror'
else:
    choice = 'B2syserror'
errgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = choice).sort_index(axis = 0, ascending = False)
myheatmap(errgrid, choice, vmax=vmax, cmap='magma_r'); 
plt.title('1d')

plt.sca(ax4)
SSgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'avgsyserror').sort_index(axis = 0, ascending = False)
myheatmap(SSgrid, "average sys error",  vmax=vmax, cmap='magma_r'); 
plt.title('1d')

plt.sca(ax4b)
SSgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'maxsyserr').sort_index(axis = 0, ascending = False)
myheatmap(SSgrid, "max sys error",  vmax=vmax, cmap='magma_r'); 
plt.title('1d')

plt.sca(ax5)
errgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = choice + '_2D').sort_index(axis = 0, ascending = False)
myheatmap(errgrid, choice, vmax=vmax, cmap='magma_r'); 
plt.title('2d')

plt.sca(ax6)
SSgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'avgsyserror_2D').sort_index(axis = 0, ascending = False)
myheatmap(SSgrid, "average sys error",  vmax=vmax, cmap='magma_r'); 
plt.title('2d')

plt.sca(ax6b)
grid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'maxsyserr_2D').sort_index(axis = 0, ascending = False)
myheatmap(grid, "max sys error",  vmax=vmax, cmap='magma_r'); 
plt.title('2d')

maxsyserr_2D


    
#plt.figure()
#plt.plot(resultsdf.Freq1,resultsdf.SNR_R2_f1 )

fig.tight_layout()


plt.sca(ax7)
syserrordiffgrid=resultsdf.pivot_table(index = 'Freq1', 
                                       columns = 'Freq2', 
                                       values = 'avgsyserror-avgsyserror_2D').sort_index(axis = 0, ascending = False)
myheatmap(syserrordiffgrid, "avgsyserror1D-avgsyserror_2D",  cmap='Spectral', vmin = -.5, vmax = .5); 

for ax in [ax1,ax2,ax3,ax4, ax4b, ax5, ax6, ax6b, ax7]:
    plt.sca(ax)
    ax.axis('equal');
    plt.xticks([res1, res2])
    plt.yticks([res1, res2])

sns.set_context('talk')


"""plt.figure()
grid=resultsdf.pivot_table(index = 'SNR_R2_f1', 
                                       columns = 'SNR_R2_f2', 
                                       values = 'maxsyserr_2D').sort_index(axis = 0, ascending = False)
myheatmap(grid, "maxsyserr_2D", cmap = 'magma', vmax = .1 );
plt.gca().axis('equal');"""

## remove diagonal elements from resultsdf for the following plots.
resultsdforiginal = resultsdf.copy()
del resultsdf
resultsdf = resultsdforiginal[resultsdforiginal.Difference != 0]

maxsyserror_to_plot = 1

plt.figure()
if n>30:
    alpha = .4
else:
    alpha = .8
if MONOMER:
    sc = plt.scatter(resultsdf.SNR_R1_f2, resultsdf.SNR_R1_f1, s=10, c = resultsdf.maxsyserr_2D*100, cmap = 'magma_r', vmax = maxsyserror_to_plot , alpha = alpha) 
    plt.xlabel('SNR_R1_f2')
    plt.ylabel('SNR_R1_f1')
else:
    sc = plt.scatter(resultsdf.SNR_R2_f2, resultsdf.SNR_R2_f1, s=10, c = resultsdf.maxsyserr_2D*100, cmap = 'magma_r', vmax = maxsyserror_to_plot , alpha = alpha) 
    plt.xlabel('SNR_R2_f2')
    plt.ylabel('SNR_R2_f1')
cbar = plt.colorbar(sc)
cbar.outline.set_visible(False)
cbar.set_label('maxsyserr_2D (%)')
plt.gca().axis('equal');


## maybe need to cut out f1 = f2
plt.figure()
if MONOMER:
    plt.plot(resultsdf.SNR_R1_f1, resultsdf.maxsyserr_2D*100, '.', alpha=.5)
    plt.xlabel('SNR_R1_f1')
else:
    plt.plot(resultsdf.SNR_R2_f1, resultsdf.maxsyserr_2D*100, '.', alpha=.5)
    plt.xlabel('SNR_R2_f1') 
plt.ylabel('maxsyserr_2D (%)')
plt.ylim(ymin=0, ymax=maxsyserror_to_plot)

plt.figure()
plt.plot(resultsdf.Freq1, resultsdf.maxsyserr_2D *100, '.', alpha=.5,)
plt.ylim(ymin=0, ymax=maxsyserror_to_plot)
plt.xlabel('Freq1 (rad/s)')
plt.ylabel('maxsyserr_2D (%)')
plt.xticks([res1, res2]);

In [ ]:
## Plot systematic error of parameters, sum of squares, as a function of the higher freq 

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize = (14,10))

ax1.plot(resultsdf.Freq2,resultsdf.avgsyserror*100, '.')
ax1.set_xlabel('Frequency 2 (rad/s)')
ax1.set_ylabel('Average Sys error (%)')
#ax1.set_title('Systematic error sum of squares')



ax2.plot(resultsdf.Freq2,(resultsdf.M1), '.')
ax2.set_ylabel('Mass 1')
ax2.set_xlabel('Frequency 2 (rad/s)')

ax3.plot(resultsdf.Freq2,(resultsdf.M2), '.')
ax3.set_ylabel('Mass 2')
ax3.set_xlabel('Frequency 2 (rad/s)')

ax4.plot(resultsdf.Freq2,(resultsdf.B2), '.')
ax4.set_ylabel(' value of damping 2')
ax4.set_xlabel('Frequency 2 (rad/s)')

plt.tight_layout()



In [ ]:
#Plots of badness of fits as a function of separation between points

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize = (14,10))

ax1.plot(resultsdf.Difference, (resultsdf.R1AmpFit), '.', color = 'silver', alpha=0.5)
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Difference Squared ($m^2$)')
ax1.set_title('R1 Amp Badness of Fit')
        
ax2.plot(resultsdf.Difference, (resultsdf.R1PhaseFit), '.', color = 'silver',alpha=0.5)
ax2.set_xlabel('Frequency Difference (rad/s)')
ax2.set_ylabel('Difference Squared ($rad^2$)')
ax2.set_title('R1 Phase Badness of Fit')     

ax3.plot(resultsdf.Difference, (resultsdf.R2AmpFit), '.', color = 'lightsteelblue',alpha=0.5)
ax3.set_xlabel('Frequency Difference (rad/s)')
ax3.set_ylabel('Difference Squared ($m^2$)')
ax3.set_title('R2 Amp Badness of Fit')
    
ax4.plot(resultsdf.Difference, (resultsdf.R2PhaseFit), '.', color = 'lightsteelblue',alpha=0.5)
ax4.set_xlabel('Frequency Difference (rad/s)')
ax4.set_ylabel('Difference Squared ($rad^2$)')
ax4.set_title('R2 Phase Badness of Fit')
        
plt.tight_layout()

#Print frequency difference with smallest badness of fit

print("R1 Amp min = ", resultsdf.iloc[resultsdf.R1AmpFit.idxmin()].Difference, "m",
     "\nR1 Phase min = ", resultsdf.iloc[resultsdf.R1PhaseFit.idxmin()].Difference, "rad",
     "\nR2 Amp min = ", resultsdf.iloc[resultsdf.R2AmpFit.idxmin()].Difference, "m",
     "\nR2 Phase min = ", resultsdf.iloc[resultsdf.R2PhaseFit.idxmin()].Difference, "rad")

In [ ]:
#Plot combined amplitude and phase Badness of Fit parameters

#Plots of badness of fits as a function of seperation between points

fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize = (14,5))

ax1.plot(resultsdf.Difference, (resultsdf.R1AmpFit)+(resultsdf.R2AmpFit), '.', color = 'silver',alpha=0.5)
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Difference Squared ($m^2$)')
ax1.set_title('Amplitude Badness of Fit')
        
ax2.plot(resultsdf.Difference, (resultsdf.R1PhaseFit)+(resultsdf.R2PhaseFit), '.', color = 'silver',alpha=0.5)
ax2.set_xlabel('Frequency Difference (rad/s)')
ax2.set_ylabel('Difference Squared ($rad^2$)')
ax2.set_title('Phase Badness of Fit')     

plt.tight_layout()

#Print frequency difference with smallest badness of fit

print("Amp min = ", resultsdf.iloc[resultsdf.R1AmpFit.idxmin()].Difference + 
      resultsdf.iloc[resultsdf.R2AmpFit.idxmin()].Difference, "m",
     "\nR1 Phase min = ", resultsdf.iloc[resultsdf.R1PhaseFit.idxmin()].Difference +
      resultsdf.iloc[resultsdf.R2PhaseFit.idxmin()].Difference, "rad")

In [ ]:
#Plots of badness of fits as a function of seperation between points

fig, (ax1) = plt.subplots(1, 1, figsize = (8,6))

ax1.plot(resultsdf.Difference, np.log(resultsdf.R1AmpFit + resultsdf.R2AmpFit + resultsdf.R1PhaseFit + resultsdf.R2PhaseFit), '.', color = 'silver',alpha=0.03)
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Log Difference Squared ($m^2$)')
ax1.set_title('Total Badness of Fit')

print("Total min = ", resultsdf.iloc[(resultsdf.R1AmpFit + resultsdf.R2AmpFit + resultsdf.R1PhaseFit + resultsdf.R2PhaseFit).idxmin()].Difference, "rad/s",)

In [ ]:
## Plots of badness of fits as a function of seperation between points

fig, (ax1) = plt.subplots(1, 1, figsize = (8,6))

ax1.plot(resultsdf.R1_phase_diff, np.log(resultsdf.R1AmpFit + resultsdf.R2AmpFit + resultsdf.R1PhaseFit + resultsdf.R2PhaseFit), '.', color = 'silver', alpha = .05)
ax1.set_xlabel('Phase Difference (rad)')
ax1.set_ylabel('Log Difference Squared ($m^2$)')
ax1.set_title('Total Badness of Fit')

print("Total min = ", resultsdf.iloc[(resultsdf.R1AmpFit + resultsdf.R2AmpFit + resultsdf.R1PhaseFit + resultsdf.R2PhaseFit).idxmin()].Difference, "rad/s")

In [ ]:
stophere

In [ ]:
#!pip install pyDOE2
import pyDOE2

In [ ]:
## Factorial design
# vals_set = [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set]
# 1 or 10

doe = pyDOE2.fullfact([2,2,2,2,2,2,2,2]) * 9 + 1
doe

In [ ]:
# factorial design.

"""# desiredfreqs = [2.1, 2.4]
#desiredfreqs = [res1,res2]
desiredfreqs = [res1,res2, 2.1, 2.4]

p = freqpoints(desiredfreqs, drive)
"""

#p = [14,15,16,29,30,31] #indices of frequencies to use
#p=[14,16]
p = range(len(drive))
print(p)


"""
try:
    del k12_set # just to make sure there's no mistake
except NameError:
    print('re-running')
"""

# options
verbose = False
move_peaks = False

#initialize
measurementdflist = []
results = []
SNRtable = []
first = True

for vals_set in doe:
    
    [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set] = vals_set
    b1_set = b1_set/10; # need underdamping for resonance
    b2_set = b2_set/10;
    vals_set = [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set]
    
    res1 = approx_res_freq(k1_set, m1_set, b1_set)
    res2 = approx_res_freq(k2_set, m2_set, b2_set)
    
    ## calculate the spectra with k12_set
    R1_amp, R1_phase, R2_amp, R2_phase, R1_real_amp, R1_im_amp, R2_real_amp, R2_im_amp = \
        calculate_spectra(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, n)
    
    ## find peaks and choose frequency locations that match
    if move_peaks:
        newp, heights = find_peaks(R2_amp, height=.015)

        if len(newp) < 1: # just use the default if you didn't find a peak
            newp = p
            print("Didn't find a peak")
        elif len(newp) < 2: # only found one peak; use the default for the other peak.
            if newp[0] != p[1]:
                p[0] = newp[0]
                print('Found one peak,' + str(newp))
            else:
                print('Found one peak, ' + str(newp)  + ' but it is the same as what I would have used anyway.')
        if len(newp) > 2: 
            print('Found too many peaks, ' + str(newp) + str(heights))
            if newp[0] <=2:
                newp = newp[1:] # remove that 
            p_df = pd.DataFrame.join(pd.DataFrame(data = newp), pd.DataFrame(data=heights))
            while len(p_df) > 2:
                minindex = p_df[['peak_heights']].idxmin()
                p_df = p_df.drop(p_df.index[minindex])

            p = p_df[[0]].transpose().values.tolist()[0]

        print(p)

    SNR_R1_f1,SNR_R2_f1, A1f1avg, A1f1std, A2f1avg, A2f1std = SNRcalc(
        drive[p[0]], vals_set=vals_set, noiselevel = noiselevel, detailed = True)
    SNR_R1_f2,SNR_R2_f2, A1f2avg, A1f2std, A2f2avg, A2f2std = SNRcalc(
        drive[p[1]], vals_set=vals_set, noiselevel = noiselevel, detailed = True)
    
    if first:
        fig, SNRax = plt.subplots()
    thistable = []
    for i in range(len(p)):
        SNR1, SNR2 = SNRcalc(drive[p[i]], vals_set=vals_set, noiselevel = noiselevel, plot = True, ax = SNRax,)
        thistable.append([k12_set, drive[p[i]], R1_amp[p[i]], R1_phase[p[i]], R2_amp[p[i]], R2_phase[p[i]], 
                      complexamp(R1_amp[p[i]],R1_phase[p[i]] ),
                      complexamp(R2_amp[p[i]], R2_phase[p[i]]),
                      SNR1, SNR2,
                     syserror(R1_amp[p[i]], R1_amp_noiseless[p[i]]),
                     (R1_phase[p[i]] - R1_phase_noiseless[p[i]]),
                     syserror(R2_amp[p[i]],R2_amp_noiseless[p[i]]),
                     (R2_phase[p[i]]-R2_phase_noiseless[p[i]]),
                     ])

    thismeasurementdf = pd.DataFrame(data = thistable, columns = ['k12', 'drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase',
                                               'R1AmpCom', 'R2AmpCom',
                                               'SNR_R1','SNR_R2',
                                               'R1Amp_syserror', 'R1Phase_diff', 'R2Amp_syserror', 'R2Phase_diff'])
    if verbose:
        display(thismeasurementdf)
    measurementdflist.append(thismeasurementdf)
    
    Zmatrix = Zmatrix2resonators(thismeasurementdf, 
                                     frequencycolumn = 'drive', 
                                     complexamplitude1 = 'R1AmpCom', 
                                     complexamplitude2 = 'R2AmpCom',
                                     dtype = np.double)
        
    #display(pd.DataFrame(Zmatrix))

    #SVD,  u and vh are 2D unitary arrays and s is a 1D array of the input matrix's singular values
    u, s, vh = np.linalg.svd(Zmatrix, full_matrices = True)

    ## 1D NULLSPACE
    # extract parameters found by SVD
    #assign variables # mass of resonator 1, mass 2, damping 1, damping 2, stiffness 1, stiffness 2, coupling stiffness, force
    [M1, M2, B1, B2, K1, K2, K12, FD] = vh[-1] # the 7th singular value is the smallest one (closest to zero)

    # normalize elements vector to the force, assuming 1D nullspace
    allelements = normalize_elements_1d_by_force([M1, M2, B1, B2, K1, K2, K12, FD], F_set)
    # recast as real, not complex # but real gets a warning
    # allelements = [thiselement.real for thiselement in allelements if thiselement.imag == 0 ]
    M1, M2, B1, B2, K1, K2, K12, FD = allelements
        
    # Alternative check: 
    # calculate how close the SVD-determined parameters are compared to the originally set parameters

    if MONOMER:
        el = [M1,  B1,  K1,  FD]
    else:
        el = [M1, M2, B1, B2, K1, K2, K12, FD]
        
    syserrors = [syserror(el[i], vals_set[i]) for i in range(len(el))]

    # Values to compare:
    # Set values: k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set
    # SVD-determined values: M1, M2, B1, B2, K1, K2, K12, FD
    K1syserror = syserror(K1,k1_set)
    K2syserror = syserror(K2,k2_set)
    K12syserror = syserror(K12,k12_set)
    B1syserror = syserror(B1,b1_set)
    B2syserror = syserror(B2,b2_set)
    FDsyserror = syserror(FD,F_set)
    M1syserror = syserror(M1,m1_set)
    M2syserror = syserror(M2,m2_set)
    avgsyserror, rmssyserror, maxsyserr = combinedsyserror(syserrors,1) # subtract 1 degrees of freedom for 1D nullspace
    """        parametererror2 = np.sqrt((K1syserror**2 + K2syserror**2 + K12syserror**2 \
                              + B1syserror**2 + B2syserror**2 + FDsyserror**2 + \
                              M1syserror**2 + M2syserror**2)/8)"""

    ### Normalize elements in 2D nullspace 
    """ # Problem: res1 formula only for weak coupling.
    [M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D] = \
        normalize_elements_to_res1_and_F_2d(vh, vals_set = vals_set)
    coefa = np.nan
    coefb = np.nan"""
    #[M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D], coefa, coefb = \
    #    normalize_elements_to_m1_set_k1_set_assuming_2d(vh)
    #[M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D], coefa, coefb = \
    #    normalize_elements_to_m1_m2_assuming_2d(vh, verbose = False, m1_set = m1_set, m2_set = m2_set)
    [M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D], coefa, coefb = \
        normalize_elements_to_m1_F_set_assuming_2d(vh, verbose = False, m1_set = m1_set, F_set = F_set)
    normalizationpair = 'm1 and F'
    if MONOMER:
        el_2D = M1_2D, B1_2D, K1_2D,  FD_2D
    else:
        el_2D = M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D 
    syserrors_2D = [syserror(el_2D[i], vals_set[i]) for i in range(len(el_2D))]

    # Values to compare:
    # Set values: k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set
    # SVD-determined values: M1, M2, B1, B2, K1, K2, K12, FD

    K1syserror_2D = syserror(K1_2D,k1_set)
    B1syserror_2D = syserror(B1_2D,b1_set)
    FDsyserror_2D = syserror(FD_2D,F_set)
    M1syserror_2D = syserror(M1_2D,m1_set)

    if MONOMER:
        K2syserror_2D = 0
        K12syserror_2D = 0
        B2syserror_2D = 0
        M2syserror_2D = 0
    else:
        K2syserror_2D = syserror(K2_2D,k2_set)
        K12syserror_2D = syserror(K12_2D,k12_set)
        B2syserror_2D = syserror(B2_2D,b2_set)
        M2syserror_2D = syserror(M2_2D,m2_set)

    avgsyserror_2D, rmssyserror_2D, maxsyserr_2D = combinedsyserror(syserrors_2D,2) # subtract 2 degrees of freedom for 2D nullspace

    if first or s[-2]>4:
        print('m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set are:')
        print(vals_set)
        print("Systematic errors for 2D result:" + str(syserrors_2D*100) + '%')
        plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase, thismeasurementdf,  K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, vals_set)
        plt.show()
        first = False
    
    # save the results (note: I mean saving to temporary kernel memory, not to harddrive)
    results.append([m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set,
                    approx_Q(m = m1_set, k = k1_set, b = b1_set),
                    approx_Q(m = m2_set, k = k2_set, b = b2_set),
                    drive[p[0]],
                      drive[p[1]],
                      drive[p[1]] - drive[p[0]],
                      R1_amp[p[0]], R1_phase[p[0]], R2_amp[p[0]], R2_phase[p[0]],
                              R1_amp[p[1]], R1_phase[p[1]], R2_amp[p[1]], R2_phase[p[1]],
                      M1, M2, B1, B2, K1, K2, K12, FD, # elements vector, from 1D SVD
                      K1syserror,
                      K2syserror,
                      K12syserror,
                      B1syserror,
                      B2syserror,
                      FDsyserror,
                      M1syserror,
                      M2syserror,
                    any(x<0 for x in el),
                      avgsyserror, rmssyserror,maxsyserr,
                     np.sum(R1_amp - curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2,   # sum of squares
                    np.sum((R1_phase - theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                    np.sum((R2_amp - curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                    np.sum((R2_phase - theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                      M1_2D, M2_2D, B1_2D, B2_2D, K1_2D, K2_2D, K12_2D, FD_2D, # elements vector, from 2D SVD
                      K1syserror_2D,
                      K2syserror_2D,
                      K12syserror_2D,
                      B1syserror_2D,
                      B2syserror_2D,
                      FDsyserror_2D,
                      M1syserror_2D,
                      M2syserror_2D,
                    any(x<0 for x in el_2D),
                      avgsyserror_2D, rmssyserror_2D,maxsyserr_2D,
                    # sum of squares
                     np.sum(R1_amp - curve1(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2, 
                    np.sum((R1_phase - theta1(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                    np.sum((R2_amp - curve2(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                    np.sum((R2_phase - theta2(drive, K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, 0))**2),
                    avgsyserror - avgsyserror_2D,
                    s[-1],s[-2], # smallest two singular values
                    SNR_R1_f1,SNR_R2_f1, # signal to noise ratio
                    A1f1avg, A1f1std, A2f1avg, A2f1std,
                    SNR_R1_f2,SNR_R2_f2,
                     A1f2avg, A1f2std, A2f2avg, A2f2std])
        

resultsdoedf = pd.DataFrame(
    data=results, 
    columns = ['m1_set', 'm2_set', 'b1_set', 'b2_set', 'k1_set', 'k2_set', 'k12_set', 'F_set', 
               'approxQ1', 'approxQ2',
               'Freq1','Freq2', 'Difference',
               'R1_amp_meas_a', 'R1_phase_meas_a', 'R2_amp_meas_a', 'R2_phase_meas_a',
               'R1_amp_meas_b', 'R1_phase_meas_b', 'R2_amp_meas_b', 'R2_phase_meas_b',
        'M1', 'M2', 'B1', 'B2', 'K1', 'K2', 'K12', 'FD', # elements vector, from 1D SVD
        'K1syserror',
        'K2syserror',
        'K12syserror',
        'B1syserror',
        'B2syserror',
        'FDsyserror',
        'M1syserror',
        'M2syserror',
               'any neg 1D',
        'avgsyserror', 'rmssyserror', 'maxsyserr',
        'R1AmpFit', 'R1PhaseFit', 'R2AmpFit', 'R2PhaseFit',   # sum of squares
        'M1_2D', 'M2_2D', 'B1_2D', 'B2_2D', 'K1_2D', 'K2_2D', 'K12_2D', 'FD_2D', # elements vector, from 2D SVD
        'K1syserror_2D',
        'K2syserror_2D',
        'K12syserror_2D',
        'B1syserror_2D',
        'B2syserror_2D',
        'FDsyserror_2D',
        'M1syserror_2D',
        'M2syserror_2D',
               'any neg 2D',
        'avgsyserror_2D', 'rmssyserror_2D', 'maxsyserr_2D',
        'R1AmpFit_2D', 'R1PhaseFit_2D', 'R2AmpFit_2D', 'R2PhaseFit_2D', # sum of squares
        'avgsyserror-avgsyserror_2D',
        'smallest singular value', 'second smallest singular value',
        'SNR_R1_f1','SNR_R2_f1',
               'A1f1avg', 'A1f1std', 'A2f1avg', 'A2f1std',
        'SNR_R1_f2','SNR_R2_f2',
                'A1f2avg', 'A1f2std', 'A2f2avg', 'A2f2std'])

resultsdoedf

In [ ]:
#save to harddrive
resultsdoedf.to_csv(r'G:\Shared drives\Horowitz Lab Notes\Horowitz, Viva - notes and files\2022-04-21resultsdoe.csv')

In [ ]:
resultsdoedf.to_pickle(r'G:\Shared drives\Horowitz Lab Notes\Horowitz, Viva - notes and files\2022-04-21resultsdoe.pkl')

In [ ]:
print('m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set are:')
print(str(vals_set))
plot_SVD_results(drive,R1_amp,R1_phase,R2_amp,R2_phase, thismeasurementdf,  K1_2D, K2_2D, K12_2D, B1_2D, B2_2D, FD_2D, M1_2D, M2_2D, vals_set)
plt.show()

In [ ]:
len(resultsdoedf[resultsdoedf['any neg 1D']])

In [ ]:
len(resultsdoedf[resultsdoedf['any neg 2D']])

In [ ]:
# Both 1d and 2d are impossible answers.
resultsdoedf[resultsdoedf['any neg 1D'] & resultsdoedf['any neg 2D']]

In [ ]:
resultsdoedfclean1 = resultsdoedf[-resultsdoedf['any neg 1D']]
resultsdoedfclean = resultsdoedfclean1[-resultsdoedfclean1['any neg 2D']]
print(len(resultsdoedfclean))

print('How could I guess that the nullspace is 1D or 2D?')

fig, (ax1, ax2) = plt.subplots(2,1, gridspec_kw={'hspace': 0}, sharex = 'all')

## Distribution of s[-2] (second smallest singular value) when 1D is better

oneDbetter = resultsdoedfclean[resultsdoedfclean.avgsyserror_2D > resultsdoedfclean.avgsyserror]
plt.sca(ax1)
oneDbetter['second smallest singular value'].hist()
plt.ylabel('1D')

## Distribution of s[-2] when 2D is better
plt.sca(ax2)
twoDbetter = resultsdoedfclean[resultsdoedfclean.avgsyserror_2D < resultsdoedfclean.avgsyserror]
twoDbetter['second smallest singular value'].hist()
plt.ylabel('nullspace 2D')

plt.xlabel('second smallest singular value');

# ==============

fig, (ax1, ax2) = plt.subplots(2,1, gridspec_kw={'hspace': 0}, sharex = 'all')

plt.sca(ax1)
plt.scatter(twoDbetter['second smallest singular value'], twoDbetter.rmssyserror_2D*100, alpha=.3)
plt.ylabel('rms syserror 2D (%)')

plt.sca(ax2)
plt.scatter(oneDbetter['second smallest singular value'], oneDbetter.rmssyserror_2D*100, alpha=.3)
plt.ylabel('rms syserror 1D (%)\n')
plt.ylim(ymax=50, ymin=0)
ax2.set_xscale('log')

plt.xlabel('second smallest singular value')


#================

fig, (ax1, ax2) = plt.subplots(2,1, gridspec_kw={'hspace': 0}, sharex = 'all')

## Distribution of s[-1] (smallest singular value) when 1D is better

plt.sca(ax1)
oneDbetter['smallest singular value'].hist()
plt.ylabel('1D')

## Distribution of s[-1] when 2D is better
plt.sca(ax2)
twoDbetter['smallest singular value'].hist()
plt.ylabel('nullspace 2D')

plt.xlabel('smallest singular value');

# ==============

fig, (ax1, ax2) = plt.subplots(2,1, gridspec_kw={'hspace': 0}, sharex = 'all')

plt.sca(ax1)
plt.scatter(twoDbetter['smallest singular value'], twoDbetter.rmssyserror_2D*100, alpha=.3)
plt.ylabel('rms syserror 2D (%)')
ax1.set_xscale('log')

plt.sca(ax2)
plt.scatter(oneDbetter['smallest singular value'], oneDbetter.rmssyserror_2D*100, alpha=.3)
plt.ylabel('rms syserror 1D (%)\n')
plt.ylim(ymax=50, ymin=0)
ax2.set_xscale('log')

plt.xlabel('smallest singular value');

plt.show()

print('I do not see any way to distinguish.')


In [ ]:
oneDbetter

In [ ]:
twoDbetter

In [ ]:
plt.scatter(oneDbetter['k2_set'], oneDbetter.rmssyserror_2D*100, alpha=.3)
plt.ylim(0,50)

In [ ]:
with pd.option_context('display.max_rows', None,):
    display(twoDbetter[twoDbetter['second smallest singular value'] > 4].transpose())

In [ ]:
stophere

In [ ]:
## vary... k12, m1, k2, SNR (using force to vary SNR), noiselevel, number of frequency points. Choose frequency points by phase.